In the previous notebooks we modified the train data.

In the following notebook we apply the same modifications to the test data in order to calculate sales predictions.

# DATA CLEANING & CREAZIONE DATASET

## IMPORT

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import Timedelta
import numpy as np
import warnings
warnings.filterwarnings("ignore")


pd.set_option('display.max_columns', None)

In [2]:
ice = pd.read_csv('./Working Code/Datasets/icecat_caratteristiche_test.csv')
kpi = pd.read_csv('./Working Code/Datasets/kpi_prodotti_volantini_test.csv')
anagrafica = pd.read_csv('./Working Code/Datasets/anagrafica_volantini_test.csv', encoding='ISO-8859-1')
storico = pd.read_csv('./Working Code/Datasets/storico_quantita_test.csv')
gfk = pd.read_csv('./Working Code/Datasets/gfk_caratteristiche_test.csv')

## DATA CLEANING

### ICE

In [3]:
ice.head()

,ART_COD,name,title,description,macro_feature_cod,macro_feature_des,feature_cod,feature_des,feature_value,feature_value_nounit,feature_value_unit_cod,feature_value_unit_des,feature_value_type
0,CANCST360L01,CST 360L-01,Candy Smart CST 360L-01 lavatrice Caricamento ...,Smart\nCandy crede nell'importanza del godersi...,1829,Gestione energetica,2494,Consumo di energia per lavaggio,0.765 kWh,0.765,173.0,kWh,number
1,CANCST360L01,CST 360L-01,Candy Smart CST 360L-01 lavatrice Caricamento ...,Smart\nCandy crede nell'importanza del godersi...,1829,Gestione energetica,8491,Tensione di ingresso AC,220 - 240 V,220 - 240,379.0,V,text
2,CANCST360L01,CST 360L-01,Candy Smart CST 360L-01 lavatrice Caricamento ...,Smart\nCandy crede nell'importanza del godersi...,1829,Gestione energetica,8484,Frequenza di ingresso AC,50 Hz,50,130.0,Hz,text
3,CANCST360L01,CST 360L-01,Candy Smart CST 360L-01 lavatrice Caricamento ...,Smart\nCandy crede nell'importanza del godersi...,1829,Gestione energetica,2034,Consumi (modalità spento),0.2 W,0.2,387.0,W,number
4,CANCST360L01,CST 360L-01,Candy Smart CST 360L-01 lavatrice Caricamento ...,Smart\nCandy crede nell'importanza del godersi...,1829,Gestione energetica,2036,Consumi (modalità stand-by),1 W,1,387.0,W,number


In [4]:
ice.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491277 entries, 0 to 491276
Data columns (total 13 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   ART_COD                 491277 non-null  object
 1   name                    491277 non-null  object
 2   title                   491277 non-null  object
 3   description             491153 non-null  object
 4   macro_feature_cod       491277 non-null  int64 
 5   macro_feature_des       491277 non-null  object
 6   feature_cod             491277 non-null  int64 
 7   feature_des             491277 non-null  object
 8   feature_value           466447 non-null  object
 9   feature_value_nounit    466444 non-null  object
 10  feature_value_unit_cod  125171 non-null  object
 11  feature_value_unit_des  125171 non-null  object
 12  feature_value_type      491277 non-null  object
dtypes: int64(2), object(11)
memory usage: 48.7+ MB


In [5]:
ice['feature_value_unit_des'].nunique()

71

In [6]:
ice.describe()

,macro_feature_cod,feature_cod
count,491277.000000,491277.000000
mean,8948.912101,12231.972417
std,7607.729523,12572.139867
min,-1.000000,5.000000
25%,5078.000000,2328.000000
50%,6890.000000,7979.000000
75%,10707.000000,17554.000000
max,77548.000000,47293.000000


In [7]:
ice.isnull().sum()

ART_COD                        0
name                           0
title                          0
description                  124
macro_feature_cod              0
macro_feature_des              0
feature_cod                    0
feature_des                    0
feature_value              24830
feature_value_nounit       24833
feature_value_unit_cod    366106
feature_value_unit_des    366106
feature_value_type             0
dtype: int64

In [8]:
ice[(ice['feature_value'].notnull()) & (ice['feature_value_nounit'].isnull())]

,ART_COD,name,title,description,macro_feature_cod,macro_feature_des,feature_cod,feature_des,feature_value,feature_value_nounit,feature_value_unit_cod,feature_value_unit_des,feature_value_type
100022,HORVIEW10LITE64,Honor View 10 Lite 6.5 Doppia SIM,Honor View 10 Lite 6.5 Doppia SIM,Sistema operativo Android 8.1 Processore Octa ...,6887,Archiviazione,33517,Tipo di slot per scheda di memoria,microSD,NaN,NaN,NaN,enumerate
100025,HORVIEW10LITE64,Honor View 10 Lite 6.5 Doppia SIM,Honor View 10 Lite 6.5 Doppia SIM,Sistema operativo Android 8.1 Processore Octa ...,6890,Collegamento in rete,1208,Versione Bluetooth,4.2,NaN,NaN,NaN,enumerate
100026,HORVIEW10LITE64,Honor View 10 Lite 6.5 Doppia SIM,Honor View 10 Lite 6.5 Doppia SIM,Sistema operativo Android 8.1 Processore Octa ...,6890,Collegamento in rete,16921,4G,Sì,NaN,NaN,NaN,enumerate


### KPI

In [9]:
kpi.head()

,CODICE_VOLANTINO,ART_COD,PRODUCT_GROUP,ITEM_ID,KPI_1,KPI_2,KPI_3,KPI_4,KPI_5
0,23MA,ACEA3155879TU,MOBILE COMPUTING,172435098,0.960614,0.577713,0.622543,0.193957,1.000000
1,23MA,AEGL6FBI48W,WASHINGMACHINES FREESTANDING,176720202,0.949306,0.757657,0.338377,0.789157,0.200890
2,23MA,AMFGTR3,CORE WEARABLES,172236545,0.423670,0.209620,0.165108,1.000000,0.720034
3,23MA,AMFGTR3GREY,CORE WEARABLES,172236545,0.423670,0.209620,0.039904,1.000000,0.720034
4,23MA,APLIPHONE13128,SMARTPHONES,171845322,0.768852,0.725024,0.726402,0.449412,1.000000


In [10]:
kpi['ITEM_ID'].replace('ND', np.nan, inplace=True)
kpi['ITEM_ID'] = kpi['ITEM_ID'].astype(float)

In [11]:
kpi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1382 entries, 0 to 1381
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CODICE_VOLANTINO  1382 non-null   object 
 1   ART_COD           1382 non-null   object 
 2   PRODUCT_GROUP     1382 non-null   object 
 3   ITEM_ID           1330 non-null   float64
 4   KPI_1             1382 non-null   float64
 5   KPI_2             1205 non-null   float64
 6   KPI_3             1224 non-null   float64
 7   KPI_4             1297 non-null   float64
 8   KPI_5             1199 non-null   float64
dtypes: float64(6), object(3)
memory usage: 97.3+ KB


In [12]:
kpi.describe()

,ITEM_ID,KPI_1,KPI_2,KPI_3,KPI_4,KPI_5
count,1.330000e+03,1382.000000,1205.000000,1.224000e+03,1297.000000,1199.000000
mean,1.794904e+08,0.859072,0.326937,2.450557e-01,0.592823,0.494002
std,5.179774e+06,0.188017,0.285078,2.458584e-01,0.366711,0.368892
min,1.461419e+08,0.423670,0.000000,2.867000e-07,0.000000,0.000000
25%,1.775740e+08,0.768852,0.000000,4.636393e-02,0.270186,0.135960
50%,1.807003e+08,0.960614,0.327734,1.574722e-01,0.574520,0.506164
75%,1.835322e+08,1.000000,0.525348,3.868058e-01,1.000000,1.000000
max,1.851389e+08,1.000000,1.000000,1.000000e+00,1.000000,1.000000


In [13]:
kpi['KPI_3'].where(kpi['KPI_3']>=0, pd.NA, inplace=True)
kpi

,CODICE_VOLANTINO,ART_COD,PRODUCT_GROUP,ITEM_ID,KPI_1,KPI_2,KPI_3,KPI_4,KPI_5
0,23MA,ACEA3155879TU,MOBILE COMPUTING,172435098.0,0.960614,0.577713,0.622543,0.193957,1.000000
1,23MA,AEGL6FBI48W,WASHINGMACHINES FREESTANDING,176720202.0,0.949306,0.757657,0.338377,0.789157,0.200890
2,23MA,AMFGTR3,CORE WEARABLES,172236545.0,0.423670,0.209620,0.165108,1.000000,0.720034
3,23MA,AMFGTR3GREY,CORE WEARABLES,172236545.0,0.423670,0.209620,0.039904,1.000000,0.720034
4,23MA,APLIPHONE13128,SMARTPHONES,171845322.0,0.768852,0.725024,0.726402,0.449412,1.000000
...,...,...,...,...,...,...,...,...,...
1377,23NC,XIARENOTE12PRO256W,SMARTPHONES,NaN,1.000000,NaN,NaN,NaN,NaN
1378,23NC,SAMS23FE128,SMARTPHONES,NaN,1.000000,NaN,NaN,NaN,NaN
1379,23NC,SAMS23FE128CR,SMARTPHONES,NaN,1.000000,NaN,NaN,NaN,NaN
1380,23NC,XIAREDMI13C128B,SMARTPHONES,NaN,1.000000,NaN,NaN,NaN,NaN


In [14]:
kpi.describe()

,ITEM_ID,KPI_1,KPI_2,KPI_3,KPI_4,KPI_5
count,1.330000e+03,1382.000000,1205.000000,1.224000e+03,1297.000000,1199.000000
mean,1.794904e+08,0.859072,0.326937,2.450557e-01,0.592823,0.494002
std,5.179774e+06,0.188017,0.285078,2.458584e-01,0.366711,0.368892
min,1.461419e+08,0.423670,0.000000,2.867000e-07,0.000000,0.000000
25%,1.775740e+08,0.768852,0.000000,4.636393e-02,0.270186,0.135960
50%,1.807003e+08,0.960614,0.327734,1.574722e-01,0.574520,0.506164
75%,1.835322e+08,1.000000,0.525348,3.868058e-01,1.000000,1.000000
max,1.851389e+08,1.000000,1.000000,1.000000e+00,1.000000,1.000000


In [15]:
kpi.isnull().sum()

CODICE_VOLANTINO      0
ART_COD               0
PRODUCT_GROUP         0
ITEM_ID              52
KPI_1                 0
KPI_2               177
KPI_3               158
KPI_4                85
KPI_5               183
dtype: int64

### ANAGRAFICA

In [3]:
display(anagrafica.head())
anagrafica.shape

,CODICE_VOLANTINO,DATA_INIZIO,DATA_FINE,NOME_CAMPAGNA,ART_COD,PREZZO_PROMO,PREZZO_LISTINO,STOCK_PZ,TIPOLOGIA_PRODOTTO,SCONTO_PERC,STOCK_ORD_IN_ARRIVO_PZ,PRODUCT_GROUP,BRAND
0,23MA,2023-11-01T00:00:00.0,2023-11-08T00:00:00.0,BASKET TASSO ZERO,AMFACTIVEB,129.99,129.99,0,Principale,0.0,0,CORE WEARABLES ...,AMAZFIT
1,23NA,2023-12-01T00:00:00.0,2023-12-06T00:00:00.0,BASKET TASSO ZERO,AMFACTIVEB,129.99,129.99,1111,Principale,0.0,0,CORE WEARABLES ...,AMAZFIT
2,23MA,2023-11-01T00:00:00.0,2023-11-08T00:00:00.0,BASKET TASSO ZERO,AMFACTIVEP,129.99,129.99,0,Sostitutivo,0.0,0,CORE WEARABLES ...,AMAZFIT
3,23NA,2023-12-01T00:00:00.0,2023-12-06T00:00:00.0,BASKET TASSO ZERO,AMFACTIVEP,129.99,129.99,962,Sostitutivo,0.0,0,CORE WEARABLES ...,AMAZFIT
4,23NC,2023-12-17T00:00:00.0,2023-12-24T00:00:00.0,NATALISSIMI TASSO ZERO,AMFBAND7,44.99,49.99,237,Principale,-10.0,0,CORE WEARABLES ...,AMAZFIT


(1382, 13)

In [4]:
anagrafica['DATA_INIZIO'] = anagrafica['DATA_INIZIO'].str.slice(0, 10)
anagrafica['DATA_FINE'] = anagrafica['DATA_FINE'].str.slice(0, 10)

In [18]:
anagrafica.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1382 entries, 0 to 1381
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   CODICE_VOLANTINO        1382 non-null   object 
 1   DATA_INIZIO             1382 non-null   object 
 2   DATA_FINE               1382 non-null   object 
 3   NOME_CAMPAGNA           1382 non-null   object 
 4   ART_COD                 1382 non-null   object 
 5   PREZZO_PROMO            1382 non-null   float64
 6   PREZZO_LISTINO          1382 non-null   float64
 7   STOCK_PZ                1382 non-null   int64  
 8   TIPOLOGIA_PRODOTTO      1382 non-null   object 
 9   SCONTO_PERC             1382 non-null   float64
 10  STOCK_ORD_IN_ARRIVO_PZ  1382 non-null   int64  
 11  PRODUCT_GROUP           1382 non-null   object 
 12  BRAND                   1382 non-null   object 
dtypes: float64(3), int64(2), object(8)
memory usage: 140.5+ KB


In [5]:
anagrafica.isnull().sum()

CODICE_VOLANTINO          0
DATA_INIZIO               0
DATA_FINE                 0
NOME_CAMPAGNA             0
ART_COD                   0
PREZZO_PROMO              0
PREZZO_LISTINO            0
STOCK_PZ                  0
TIPOLOGIA_PRODOTTO        0
SCONTO_PERC               0
STOCK_ORD_IN_ARRIVO_PZ    0
PRODUCT_GROUP             0
BRAND                     0
dtype: int64

In [20]:
anagrafica.describe()

,PREZZO_PROMO,PREZZO_LISTINO,STOCK_PZ,SCONTO_PERC,STOCK_ORD_IN_ARRIVO_PZ
count,1382.000000,1382.000000,1382.000000,1382.000000,1382.000000
mean,453.479631,594.580528,717.501447,-24.398915,81.026049
std,369.127898,467.894516,1832.580655,14.699590,207.379155
min,24.990000,0.000000,-15117.000000,-83.330000,0.000000
25%,179.990000,269.900000,53.000000,-35.840000,1.000000
50%,349.900000,459.000000,310.000000,-25.000000,7.000000
75%,599.900000,829.900000,885.750000,-11.120000,45.000000
max,2899.000000,4299.000000,19826.000000,35.730000,1831.000000


In [21]:
anagrafica['SCONTO_PERC'] = anagrafica['SCONTO_PERC'].abs()

In [22]:
# REPLACE 0 WITH NAN IN PREZZO_LISTINO

anagrafica['PREZZO_LISTINO'].replace(0, np.nan, inplace=True)

In [23]:
# RETRIEVE MISSING PREZZO_LISTINO BY AVERAGING PRICES WITH THE SAME ART_CODE

for i in range(anagrafica.shape[0]):
    if pd.isna(anagrafica['PREZZO_LISTINO'][i]):
        df = anagrafica[anagrafica['ART_COD'] == anagrafica['ART_COD'][i]].reset_index()
        if pd.notnull(df['PREZZO_LISTINO'].mean()):
            anagrafica['PREZZO_LISTINO'][i] = df['PREZZO_LISTINO'].mean()

In [24]:
# CALCULATE SCONTO_PERC WHERE I HAVE BOTH PREZZO_LISTINO AND PREZZO_PROMO

condizione = (
    pd.notnull(anagrafica['PREZZO_LISTINO']) &
    pd.notnull(anagrafica['PREZZO_PROMO']) &
    pd.isnull(anagrafica['SCONTO_PERC']) &
    (anagrafica['PREZZO_PROMO'] <= anagrafica['PREZZO_LISTINO'])
)

anagrafica['SCONTO_PERC'] = np.where(
    condizione,
    ((anagrafica['PREZZO_LISTINO'] - anagrafica['PREZZO_PROMO']) / anagrafica['PREZZO_LISTINO']) * 100,
    anagrafica['SCONTO_PERC']
)

In [25]:
# CALCULATE PREZZO_PROMO WHERE I HAVE BOTH PREZZO_LISTINO AND SCONTO_PERC

condizione = (
    pd.notnull(anagrafica['PREZZO_LISTINO']) &
    pd.isnull(anagrafica['PREZZO_PROMO']) &
    pd.notnull(anagrafica['SCONTO_PERC'])
)

anagrafica['PREZZO_PROMO'] = np.where(
    condizione,
    anagrafica['PREZZO_LISTINO'] * (1 - (anagrafica['SCONTO_PERC'])/100),
    anagrafica['PREZZO_PROMO']
)

In [26]:
anagrafica.isnull().sum()

CODICE_VOLANTINO          0
DATA_INIZIO               0
DATA_FINE                 0
NOME_CAMPAGNA             0
ART_COD                   0
PREZZO_PROMO              0
PREZZO_LISTINO            0
STOCK_PZ                  0
TIPOLOGIA_PRODOTTO        0
SCONTO_PERC               0
STOCK_ORD_IN_ARRIVO_PZ    0
PRODUCT_GROUP             0
BRAND                     0
dtype: int64

In [27]:
x = list(anagrafica[anagrafica['PREZZO_LISTINO'].isnull()]['ART_COD'].unique())  

x is a list containing the 'ART_COD' of all the product that have a 'PREZZO_LISTINO' null.

We will use this list later.

In [28]:
anagrafica[anagrafica['SCONTO_PERC']<1]

,CODICE_VOLANTINO,DATA_INIZIO,DATA_FINE,NOME_CAMPAGNA,ART_COD,PREZZO_PROMO,PREZZO_LISTINO,STOCK_PZ,TIPOLOGIA_PRODOTTO,SCONTO_PERC,STOCK_ORD_IN_ARRIVO_PZ,PRODUCT_GROUP,BRAND
0,23MA,2023-11-01,2023-11-08,BASKET TASSO ZERO,AMFACTIVEB,129.99,129.99,0,Principale,0.00,0,CORE WEARABLES ...,AMAZFIT
1,23NA,2023-12-01,2023-12-06,BASKET TASSO ZERO,AMFACTIVEB,129.99,129.99,1111,Principale,0.00,0,CORE WEARABLES ...,AMAZFIT
2,23MA,2023-11-01,2023-11-08,BASKET TASSO ZERO,AMFACTIVEP,129.99,129.99,0,Sostitutivo,0.00,0,CORE WEARABLES ...,AMAZFIT
3,23NA,2023-12-01,2023-12-06,BASKET TASSO ZERO,AMFACTIVEP,129.99,129.99,962,Sostitutivo,0.00,0,CORE WEARABLES ...,AMAZFIT
7,23NA,2023-12-01,2023-12-06,BASKET TASSO ZERO,AMFBIP5,89.99,89.99,-98,Principale,0.00,0,CORE WEARABLES ...,AMAZFIT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1181,23NA,2023-12-01,2023-12-06,BASKET TASSO ZERO,MOTRAZR40ULTRA256,1199.00,1199.00,82,Principale,0.00,1,SMARTPHONES ...,MOTOROLA
1325,23NA,2023-12-01,2023-12-06,BASKET TASSO ZERO,XIA13TPRO12512B,899.90,899.90,49,Sostitutivo,0.00,1,SMARTPHONES ...,XIAOMI
1342,23NC,2023-12-17,2023-12-24,NATALISSIMI TASSO ZERO,XIAREDMI13C128,159.99,159.90,-4,Principale,0.06,0,SMARTPHONES ...,XIAOMI
1343,23NC,2023-12-17,2023-12-24,NATALISSIMI TASSO ZERO,XIAREDMI13C128G,159.99,159.90,0,Sostitutivo,0.06,0,SMARTPHONES ...,XIAOMI


In [29]:
anagrafica.loc[anagrafica['SCONTO_PERC']<1, 'SCONTO_PERC'] *= 100

In [30]:
anagrafica.describe()

,PREZZO_PROMO,PREZZO_LISTINO,STOCK_PZ,SCONTO_PERC,STOCK_ORD_IN_ARRIVO_PZ
count,1382.000000,1382.000000,1382.000000,1382.000000,1382.000000
mean,453.479631,595.520470,717.501447,25.096715,81.026049
std,369.127898,468.003964,1832.580655,14.571946,207.379155
min,24.990000,24.990000,-15117.000000,0.000000,0.000000
25%,179.990000,269.900000,53.000000,12.500000,1.000000
50%,349.900000,459.000000,310.000000,25.020000,7.000000
75%,599.900000,829.900000,885.750000,35.840000,45.000000
max,2899.000000,4299.000000,19826.000000,90.000000,1831.000000


In [31]:
anagrafica.isnull().sum()

CODICE_VOLANTINO          0
DATA_INIZIO               0
DATA_FINE                 0
NOME_CAMPAGNA             0
ART_COD                   0
PREZZO_PROMO              0
PREZZO_LISTINO            0
STOCK_PZ                  0
TIPOLOGIA_PRODOTTO        0
SCONTO_PERC               0
STOCK_ORD_IN_ARRIVO_PZ    0
PRODUCT_GROUP             0
BRAND                     0
dtype: int64

In [32]:
anagrafica[anagrafica['STOCK_PZ'] < 0]

,CODICE_VOLANTINO,DATA_INIZIO,DATA_FINE,NOME_CAMPAGNA,ART_COD,PREZZO_PROMO,PREZZO_LISTINO,STOCK_PZ,TIPOLOGIA_PRODOTTO,SCONTO_PERC,STOCK_ORD_IN_ARRIVO_PZ,PRODUCT_GROUP,BRAND
7,23NA,2023-12-01,2023-12-06,BASKET TASSO ZERO,AMFBIP5,89.99,89.99,-98,Principale,0.00,0,CORE WEARABLES ...,AMAZFIT
8,23NA,2023-12-01,2023-12-06,BASKET TASSO ZERO,AMFBIP5W,89.99,89.99,-46,Sostitutivo,0.00,0,CORE WEARABLES ...,AMAZFIT
152,23NA,2023-12-01,2023-12-06,BASKET TASSO ZERO,XIAWATCH2PROBT,249.90,269.90,-4,Principale,7.41,0,CORE WEARABLES ...,XIAOMI
170,23NA,2023-12-01,2023-12-06,BASKET TASSO ZERO,APLMLXY3TA,1199.00,1349.00,-8,Sostitutivo,11.12,4,MOBILE COMPUTING ...,APPLE
171,23NC,2023-12-17,2023-12-24,NATALISSIMI TASSO ZERO,APLMLXY3TA,1199.00,1349.00,-21,Sostitutivo,11.12,6,MOBILE COMPUTING ...,APPLE
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1377,23NC,2023-12-17,2023-12-24,NATALISSIMI TASSO ZERO,XIARENOTE12PRO256GR,249.90,399.90,-44,Sostitutivo,37.51,0,SMARTPHONES ...,XIAOMI
1378,23MC,2023-11-16,2023-11-22,BLACK FRIDAY 2 - TASSO ZERO,XIARENOTE12PRO256GR,249.90,399.90,-19,Sostitutivo,37.51,0,SMARTPHONES ...,XIAOMI
1379,23NA,2023-12-01,2023-12-06,BASKET TASSO ZERO,XIARENOTE12PRO256W,249.90,399.90,-42,Sostitutivo,37.51,0,SMARTPHONES ...,XIAOMI
1380,23NC,2023-12-17,2023-12-24,NATALISSIMI TASSO ZERO,XIARENOTE12PRO256W,249.90,399.90,-51,Sostitutivo,37.51,0,SMARTPHONES ...,XIAOMI


In [33]:
anagrafica['STOCK_PZ'] = anagrafica['STOCK_PZ'].mask(anagrafica['STOCK_PZ'] < 0, 0)
anagrafica['STOCK_PZ'].fillna(0, inplace=True)

In [34]:
anagrafica = anagrafica.drop('STOCK_ORD_IN_ARRIVO_PZ', axis=1)

In [35]:
anagrafica.describe()

,PREZZO_PROMO,PREZZO_LISTINO,STOCK_PZ,SCONTO_PERC
count,1382.000000,1382.000000,1382.000000,1382.000000
mean,453.479631,595.520470,793.414616,25.096715
std,369.127898,468.003964,1552.472690,14.571946
min,24.990000,24.990000,0.000000,0.000000
25%,179.990000,269.900000,53.000000,12.500000
50%,349.900000,459.000000,310.000000,25.020000
75%,599.900000,829.900000,885.750000,35.840000
max,2899.000000,4299.000000,19826.000000,90.000000


In [36]:
anagrafica.isnull().sum()

CODICE_VOLANTINO      0
DATA_INIZIO           0
DATA_FINE             0
NOME_CAMPAGNA         0
ART_COD               0
PREZZO_PROMO          0
PREZZO_LISTINO        0
STOCK_PZ              0
TIPOLOGIA_PRODOTTO    0
SCONTO_PERC           0
PRODUCT_GROUP         0
BRAND                 0
dtype: int64

In [37]:
anagrafica.shape

(1382, 12)

### STORICO QUANTITA'

In [38]:
storico.head()

,CODICE_VOLANTINO,GIORNO,QTA,ART_COD,FATTURATO
0,23MA,2023-10-03,27,OOPA785G8128,5504.00
1,23MA,2023-09-24,1,AMFGTR3GREY,122.94
2,23MA,2023-09-23,22,MOTE13GR,1781.39
3,23MA,2023-09-29,6,XIAREDMINOTE12PRO+B,1951.98
4,23MA,2023-09-07,2,REXEW2F5W82,702.46


In [39]:
storico.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42918 entries, 0 to 42917
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CODICE_VOLANTINO  42918 non-null  object 
 1   GIORNO            42918 non-null  object 
 2   QTA               42918 non-null  int64  
 3   ART_COD           42918 non-null  object 
 4   FATTURATO         26222 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 1.6+ MB


In [40]:
storico = storico.groupby(['CODICE_VOLANTINO', 'ART_COD']).agg({'QTA': 'sum', 'FATTURATO': 'sum'}).reset_index()

In [41]:
storico.head()

,CODICE_VOLANTINO,ART_COD,QTA,FATTURATO
0,23MA,ACEA3155879TU,347,171665.89
1,23MA,ACEANV155154ZC,0,0.00
2,23MA,ACECB3153HC322,0,0.00
3,23MA,AEGL6FBI48W,64,28524.40
4,23MA,AMFACTIVEB,0,0.00


In [42]:
storico.isnull().sum()

CODICE_VOLANTINO    0
ART_COD             0
QTA                 0
FATTURATO           0
dtype: int64

### RECUPERO PREZZI LISTINI IN ANAGRAFICA

In [43]:
prezzi_listini_mancanti = {}
for i in x:   # BEFORE WE CREATED THE LIST X CONTAINING ALL THE 'ART_COD' THAT HAVE E 'PREZZO_LISTINO' NULL.
    for index in range(storico.shape[0]):
        if storico['ART_COD'][index] == i:
            prezzo = storico['FATTURATO'][index] / storico['QTA'][index]
            prezzi_listini_mancanti[i] = prezzo



In [44]:
# INSERT IN THE 'anagrafica' DATAFRAME THE 'PREZZO_LISTINO' VALUES THAT WE HAVE FOUND

for index in range(anagrafica.shape[0]):
    if anagrafica['ART_COD'][index] in prezzi_listini_mancanti.keys() and pd.isnull(anagrafica['PREZZO_LISTINO'][index]):
        anagrafica['PREZZO_LISTINO'][index] = prezzi_listini_mancanti[anagrafica['ART_COD'][index]]


In [45]:
# CALCULATE 'SCONTO_PERC' WHERE I HAVE BOTH 'PREZZO_LISTINO' AND 'PREZZO_PROMO'

condizione = (
    pd.notnull(anagrafica['PREZZO_LISTINO']) &
    pd.notnull(anagrafica['PREZZO_PROMO']) &
    pd.isnull(anagrafica['SCONTO_PERC']) &
    (anagrafica['PREZZO_PROMO'] <= anagrafica['PREZZO_LISTINO'])
)

anagrafica['SCONTO_PERC'] = np.where(
    condizione,
    ((anagrafica['PREZZO_LISTINO'] - anagrafica['PREZZO_PROMO']) / anagrafica['PREZZO_LISTINO']) * 100,
    anagrafica['SCONTO_PERC']
)

In [46]:
# SET ALL 'PREZZO_PROMO' GREATER THAN 'PREZZO_LISTINO' AS NULL

anagrafica.loc[anagrafica['PREZZO_PROMO'] > anagrafica['PREZZO_LISTINO'], 'PREZZO_PROMO'] = np.nan
anagrafica.isnull().sum()

CODICE_VOLANTINO       0
DATA_INIZIO            0
DATA_FINE              0
NOME_CAMPAGNA          0
ART_COD                0
PREZZO_PROMO          31
PREZZO_LISTINO         0
STOCK_PZ               0
TIPOLOGIA_PRODOTTO     0
SCONTO_PERC            0
PRODUCT_GROUP          0
BRAND                  0
dtype: int64

In [47]:
# RECOVERY 'PREZZO_PROMO' DIVIDING 'FATTURATO' BY 'QTA'

for index in range(anagrafica.shape[0]):
    if pd.isnull(anagrafica['PREZZO_PROMO'][index]):
        for element in range(storico.shape[0]):
            if anagrafica['ART_COD'][index] == storico['ART_COD'][element] and anagrafica['CODICE_VOLANTINO'][index] == storico['CODICE_VOLANTINO'][element]:
                prezzo = storico['FATTURATO'][element] / storico['QTA'][element]
                anagrafica['PREZZO_PROMO'][index] = prezzo

In [48]:
anagrafica.isnull().sum()

CODICE_VOLANTINO       0
DATA_INIZIO            0
DATA_FINE              0
NOME_CAMPAGNA          0
ART_COD                0
PREZZO_PROMO          17
PREZZO_LISTINO         0
STOCK_PZ               0
TIPOLOGIA_PRODOTTO     0
SCONTO_PERC            0
PRODUCT_GROUP          0
BRAND                  0
dtype: int64

In [49]:
# CALCULATE AGAIN 'SCONTO_PERC' WHERE I HAVE BOTH 'PREZZO_LISTINO' AND 'PREZZO_PROMO'

condizione = (
    pd.notnull(anagrafica['PREZZO_LISTINO']) &
    pd.notnull(anagrafica['PREZZO_PROMO']) &
    pd.isnull(anagrafica['SCONTO_PERC']) &
    (anagrafica['PREZZO_PROMO'] <= anagrafica['PREZZO_LISTINO'])
)

anagrafica['SCONTO_PERC'] = np.where(
    condizione,
    ((anagrafica['PREZZO_LISTINO'] - anagrafica['PREZZO_PROMO']) / anagrafica['PREZZO_LISTINO']) * 100,
    anagrafica['SCONTO_PERC']
)

In [50]:
anagrafica.isnull().sum()

CODICE_VOLANTINO       0
DATA_INIZIO            0
DATA_FINE              0
NOME_CAMPAGNA          0
ART_COD                0
PREZZO_PROMO          17
PREZZO_LISTINO         0
STOCK_PZ               0
TIPOLOGIA_PRODOTTO     0
SCONTO_PERC            0
PRODUCT_GROUP          0
BRAND                  0
dtype: int64

In [51]:
# CALCULATE 'PREZZO_PROMO' WHERE I HAVE BOTH 'PREZZO_LISTINO' AND 'SCONTO_PERC'

condizione = (
    pd.notnull(anagrafica['PREZZO_LISTINO']) &
    pd.isnull(anagrafica['PREZZO_PROMO']) &
    pd.notnull(anagrafica['SCONTO_PERC'])
)

anagrafica['PREZZO_PROMO'] = np.where(
    condizione,
    anagrafica['PREZZO_LISTINO'] * (1 - (anagrafica['SCONTO_PERC'])/100),
    anagrafica['PREZZO_PROMO']
)

In [52]:
anagrafica.isnull().sum()

CODICE_VOLANTINO      0
DATA_INIZIO           0
DATA_FINE             0
NOME_CAMPAGNA         0
ART_COD               0
PREZZO_PROMO          0
PREZZO_LISTINO        0
STOCK_PZ              0
TIPOLOGIA_PRODOTTO    0
SCONTO_PERC           0
PRODUCT_GROUP         0
BRAND                 0
dtype: int64

In [53]:
anagrafica.describe().T

,count,mean,std,min,25%,50%,75%,max
PREZZO_PROMO,1382.0,351.347381,1010.159009,-9720.927,169.99,349.00,599.00,2899.0
PREZZO_LISTINO,1382.0,595.520470,468.003964,24.990,269.90,459.00,829.90,4299.0
STOCK_PZ,1382.0,793.414616,1552.472690,0.000,53.00,310.00,885.75,19826.0
SCONTO_PERC,1382.0,25.096715,14.571946,0.000,12.50,25.02,35.84,90.0


In [54]:
anagrafica['PREZZO_PROMO'] = np.where(anagrafica['PREZZO_PROMO']<0, 
                                      anagrafica['PREZZO_LISTINO'] * (1 - anagrafica['SCONTO_PERC'] / 100), 
                                      anagrafica['PREZZO_PROMO'])


In [55]:
anagrafica.shape

(1382, 12)

### GFK

In [56]:
gfk.head()

,ITEM_ID,CARATTERISTICA_01,CARATTERISTICA_02,CARATTERISTICA_05,CARATTERISTICA_04,CARATTERISTICA_03,CARATTERISTICA_06,CARATTERISTICA_07,CARATTERISTICA_08
0,136498792.0,GPU MODEL RAD 520,PROCESSOR A9-SERIES,OS VERSION WIN10 HOME,STORAGE IN GB 1000,RAM IN GB 8,DISPLAY SIZE 15.6,CONVERTIBLE NO,ND
1,126394531.0,SEGMENTS NOTEBOOK,PROCESSOR CELERON N,OS VERSION WIN10 HOME,STORAGE IN GB 500,RAM IN GB 4,DISPLAY SIZE 15.6,CONVERTIBLE NO,ND
2,125136538.0,SEGMENTS NOTEBOOK,PROCESSOR CORE I5,OS VERSION MACOS HIGH SIER,STORAGE IN GB 128,RAM IN GB 8,DISPLAY SIZE 13.3,CONVERTIBLE NO,ND
3,124814100.0,SEGMENTS NOTEBOOK,PROCESSOR CORE I5,OS VERSION MACOS SIERRA,STORAGE IN GB 128,RAM IN GB 8,DISPLAY SIZE 13.3,CONVERTIBLE NO,ND
4,133558471.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
gfk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3267 entries, 0 to 3266
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ITEM_ID            3266 non-null   float64
 1   CARATTERISTICA_01  2923 non-null   object 
 2   CARATTERISTICA_02  2923 non-null   object 
 3   CARATTERISTICA_05  2923 non-null   object 
 4   CARATTERISTICA_04  2923 non-null   object 
 5   CARATTERISTICA_03  2923 non-null   object 
 6   CARATTERISTICA_06  2923 non-null   object 
 7   CARATTERISTICA_07  2923 non-null   object 
 8   CARATTERISTICA_08  2923 non-null   object 
dtypes: float64(1), object(8)
memory usage: 229.8+ KB


In [58]:
gfk.isnull().sum()

ITEM_ID                1
CARATTERISTICA_01    344
CARATTERISTICA_02    344
CARATTERISTICA_05    344
CARATTERISTICA_04    344
CARATTERISTICA_03    344
CARATTERISTICA_06    344
CARATTERISTICA_07    344
CARATTERISTICA_08    344
dtype: int64

In [59]:
gfk = gfk.dropna() # WE USED dropna() BECAUSE THERE WERE 344 ROWS WITH ONLY NULL VALUES
gfk.replace('ND', np.nan, inplace=True)
gfk.drop('CARATTERISTICA_08', axis=1, inplace=True) #DROP THIS COLUMN BECAUSE THERE ARE ONLY NULL VALUES


## UNIONE

In [60]:
df1 = pd.merge(anagrafica, kpi, on = ["ART_COD", "CODICE_VOLANTINO"])

In [61]:
df1.head()

,CODICE_VOLANTINO,DATA_INIZIO,DATA_FINE,NOME_CAMPAGNA,ART_COD,PREZZO_PROMO,PREZZO_LISTINO,STOCK_PZ,TIPOLOGIA_PRODOTTO,SCONTO_PERC,PRODUCT_GROUP_x,BRAND,PRODUCT_GROUP_y,ITEM_ID,KPI_1,KPI_2,KPI_3,KPI_4,KPI_5
0,23MA,2023-11-01,2023-11-08,BASKET TASSO ZERO,AMFACTIVEB,129.99,129.99,0,Principale,0.0,CORE WEARABLES ...,AMAZFIT,CORE WEARABLES,184518908.0,0.42367,NaN,NaN,1.0,NaN
1,23NA,2023-12-01,2023-12-06,BASKET TASSO ZERO,AMFACTIVEB,129.99,129.99,1111,Principale,0.0,CORE WEARABLES ...,AMAZFIT,CORE WEARABLES,184518908.0,1.00000,NaN,0.046811,1.0,0.756182
2,23MA,2023-11-01,2023-11-08,BASKET TASSO ZERO,AMFACTIVEP,129.99,129.99,0,Sostitutivo,0.0,CORE WEARABLES ...,AMAZFIT,CORE WEARABLES,184518908.0,0.42367,NaN,NaN,1.0,NaN
3,23NA,2023-12-01,2023-12-06,BASKET TASSO ZERO,AMFACTIVEP,129.99,129.99,962,Sostitutivo,0.0,CORE WEARABLES ...,AMAZFIT,CORE WEARABLES,184518908.0,1.00000,NaN,0.018919,1.0,0.756182
4,23NC,2023-12-17,2023-12-24,NATALISSIMI TASSO ZERO,AMFBAND7,44.99,49.99,237,Principale,10.0,CORE WEARABLES ...,AMAZFIT,CORE WEARABLES,177507779.0,1.00000,0.252783,0.061233,1.0,0.068461


In [62]:
df1 = pd.merge(df1, storico, on = ["ART_COD", "CODICE_VOLANTINO"])
df1 = df1.rename(columns={'QTA' : 'QTA_storico'})
df1 = df1.rename(columns={'FATTURATO' : 'FATTURATO_storico'})

In [63]:
df1.head()

,CODICE_VOLANTINO,DATA_INIZIO,DATA_FINE,NOME_CAMPAGNA,ART_COD,PREZZO_PROMO,PREZZO_LISTINO,STOCK_PZ,TIPOLOGIA_PRODOTTO,SCONTO_PERC,PRODUCT_GROUP_x,BRAND,PRODUCT_GROUP_y,ITEM_ID,KPI_1,KPI_2,KPI_3,KPI_4,KPI_5,QTA_storico,FATTURATO_storico
0,23MA,2023-11-01,2023-11-08,BASKET TASSO ZERO,AMFACTIVEB,129.99,129.99,0,Principale,0.0,CORE WEARABLES ...,AMAZFIT,CORE WEARABLES,184518908.0,0.42367,NaN,NaN,1.0,NaN,0,0.00
1,23NA,2023-12-01,2023-12-06,BASKET TASSO ZERO,AMFACTIVEB,129.99,129.99,1111,Principale,0.0,CORE WEARABLES ...,AMAZFIT,CORE WEARABLES,184518908.0,1.00000,NaN,0.046811,1.0,0.756182,15,1593.97
2,23MA,2023-11-01,2023-11-08,BASKET TASSO ZERO,AMFACTIVEP,129.99,129.99,0,Sostitutivo,0.0,CORE WEARABLES ...,AMAZFIT,CORE WEARABLES,184518908.0,0.42367,NaN,NaN,1.0,NaN,0,0.00
3,23NA,2023-12-01,2023-12-06,BASKET TASSO ZERO,AMFACTIVEP,129.99,129.99,962,Sostitutivo,0.0,CORE WEARABLES ...,AMAZFIT,CORE WEARABLES,184518908.0,1.00000,NaN,0.018919,1.0,0.756182,7,745.84
4,23NC,2023-12-17,2023-12-24,NATALISSIMI TASSO ZERO,AMFBAND7,44.99,49.99,237,Principale,10.0,CORE WEARABLES ...,AMAZFIT,CORE WEARABLES,177507779.0,1.00000,0.252783,0.061233,1.0,0.068461,63,2442.52


In [64]:
df1 = df1.rename(columns={'PRODUCT_GROUP_x' : 'PRODUCT_GROUP'})
df1.drop(['PRODUCT_GROUP_y'], inplace = True, axis = 1)

In [65]:
df1.columns

Index(['CODICE_VOLANTINO', 'DATA_INIZIO', 'DATA_FINE', 'NOME_CAMPAGNA',
       'ART_COD', 'PREZZO_PROMO', 'PREZZO_LISTINO', 'STOCK_PZ',
       'TIPOLOGIA_PRODOTTO', 'SCONTO_PERC', 'PRODUCT_GROUP', 'BRAND',
       'ITEM_ID', 'KPI_1', 'KPI_2', 'KPI_3', 'KPI_4', 'KPI_5', 'QTA_storico',
       'FATTURATO_storico'],
      dtype='object')

In [66]:
df1.isnull().sum()

CODICE_VOLANTINO        0
DATA_INIZIO             0
DATA_FINE               0
NOME_CAMPAGNA           0
ART_COD                 0
PREZZO_PROMO            0
PREZZO_LISTINO          0
STOCK_PZ                0
TIPOLOGIA_PRODOTTO      0
SCONTO_PERC             0
PRODUCT_GROUP           0
BRAND                   0
ITEM_ID                52
KPI_1                   0
KPI_2                 177
KPI_3                 158
KPI_4                  85
KPI_5                 183
QTA_storico             0
FATTURATO_storico       0
dtype: int64

In [67]:
df1.shape

(1382, 20)

## LAVORO SUL DATASET

In [68]:
df1.head()

,CODICE_VOLANTINO,DATA_INIZIO,DATA_FINE,NOME_CAMPAGNA,ART_COD,PREZZO_PROMO,PREZZO_LISTINO,STOCK_PZ,TIPOLOGIA_PRODOTTO,SCONTO_PERC,PRODUCT_GROUP,BRAND,ITEM_ID,KPI_1,KPI_2,KPI_3,KPI_4,KPI_5,QTA_storico,FATTURATO_storico
0,23MA,2023-11-01,2023-11-08,BASKET TASSO ZERO,AMFACTIVEB,129.99,129.99,0,Principale,0.0,CORE WEARABLES ...,AMAZFIT,184518908.0,0.42367,NaN,NaN,1.0,NaN,0,0.00
1,23NA,2023-12-01,2023-12-06,BASKET TASSO ZERO,AMFACTIVEB,129.99,129.99,1111,Principale,0.0,CORE WEARABLES ...,AMAZFIT,184518908.0,1.00000,NaN,0.046811,1.0,0.756182,15,1593.97
2,23MA,2023-11-01,2023-11-08,BASKET TASSO ZERO,AMFACTIVEP,129.99,129.99,0,Sostitutivo,0.0,CORE WEARABLES ...,AMAZFIT,184518908.0,0.42367,NaN,NaN,1.0,NaN,0,0.00
3,23NA,2023-12-01,2023-12-06,BASKET TASSO ZERO,AMFACTIVEP,129.99,129.99,962,Sostitutivo,0.0,CORE WEARABLES ...,AMAZFIT,184518908.0,1.00000,NaN,0.018919,1.0,0.756182,7,745.84
4,23NC,2023-12-17,2023-12-24,NATALISSIMI TASSO ZERO,AMFBAND7,44.99,49.99,237,Principale,10.0,CORE WEARABLES ...,AMAZFIT,177507779.0,1.00000,0.252783,0.061233,1.0,0.068461,63,2442.52


In [69]:
df1['DATA_INIZIO'] = pd.to_datetime(df1['DATA_INIZIO'])
df1['DATA_FINE'] = pd.to_datetime(df1['DATA_FINE'])

In [70]:
df1['DURATA_VOLANTINO_IN_GIORNI'] = df1['DATA_FINE'] - df1['DATA_INIZIO']
df1['DURATA_VOLANTINO_IN_GIORNI'] = df1['DURATA_VOLANTINO_IN_GIORNI'].apply(lambda x: x.days)

In [71]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1382 entries, 0 to 1381
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   CODICE_VOLANTINO            1382 non-null   object        
 1   DATA_INIZIO                 1382 non-null   datetime64[ns]
 2   DATA_FINE                   1382 non-null   datetime64[ns]
 3   NOME_CAMPAGNA               1382 non-null   object        
 4   ART_COD                     1382 non-null   object        
 5   PREZZO_PROMO                1382 non-null   float64       
 6   PREZZO_LISTINO              1382 non-null   float64       
 7   STOCK_PZ                    1382 non-null   int64         
 8   TIPOLOGIA_PRODOTTO          1382 non-null   object        
 9   SCONTO_PERC                 1382 non-null   float64       
 10  PRODUCT_GROUP               1382 non-null   object        
 11  BRAND                       1382 non-null   object      

In [72]:
df1.isnull().sum()

CODICE_VOLANTINO                0
DATA_INIZIO                     0
DATA_FINE                       0
NOME_CAMPAGNA                   0
ART_COD                         0
PREZZO_PROMO                    0
PREZZO_LISTINO                  0
STOCK_PZ                        0
TIPOLOGIA_PRODOTTO              0
SCONTO_PERC                     0
PRODUCT_GROUP                   0
BRAND                           0
ITEM_ID                        52
KPI_1                           0
KPI_2                         177
KPI_3                         158
KPI_4                          85
KPI_5                         183
QTA_storico                     0
FATTURATO_storico               0
DURATA_VOLANTINO_IN_GIORNI      0
dtype: int64

In [73]:
df1['KPI_2'].fillna(0, inplace=True)
df1['KPI_3'].fillna(0, inplace=True)
df1['KPI_4'].fillna(0, inplace=True)
df1['KPI_5'].fillna(0, inplace=True)

In [74]:
df1.describe().T

,count,mean,min,25%,50%,75%,max,std
DATA_INIZIO,1382,2023-11-25 10:52:16.324167936,2023-11-01 00:00:00,2023-11-16 00:00:00,2023-11-23 00:00:00,2023-12-07 00:00:00,2023-12-17 00:00:00,NaN
DATA_FINE,1382,2023-12-02 04:56:57.655571712,2023-11-08 00:00:00,2023-11-22 00:00:00,2023-11-30 00:00:00,2023-12-16 00:00:00,2023-12-24 00:00:00,NaN
PREZZO_PROMO,1382.0,449.316139,24.99,179.99,349.9,599.9,2899.0,363.108602
PREZZO_LISTINO,1382.0,595.52047,24.99,269.9,459.0,829.9,4299.0,468.003964
STOCK_PZ,1382.0,793.414616,0.0,53.0,310.0,885.75,19826.0,1552.47269
SCONTO_PERC,1382.0,25.096715,0.0,12.5,25.02,35.84,90.0,14.571946
ITEM_ID,1330.0,179490411.68797,146141911.0,177574023.0,180700321.0,183532152.25,185138937.0,5179774.263373
KPI_1,1382.0,0.859072,0.42367,0.768852,0.960614,1.0,1.0,0.188017
KPI_2,1382.0,0.285064,0.0,0.0,0.271409,0.502334,1.0,0.287747
KPI_3,1382.0,0.217039,0.0,0.018255,0.118825,0.35151,1.0,0.244163


In [75]:
df1.shape

(1382, 21)

# DIVISONE PER PRODUCT GROUPS ED UTILIZZO DEL DF GKF

## PREPARAZIONE

In [76]:
df1.head()

,CODICE_VOLANTINO,DATA_INIZIO,DATA_FINE,NOME_CAMPAGNA,ART_COD,PREZZO_PROMO,PREZZO_LISTINO,STOCK_PZ,TIPOLOGIA_PRODOTTO,SCONTO_PERC,PRODUCT_GROUP,BRAND,ITEM_ID,KPI_1,KPI_2,KPI_3,KPI_4,KPI_5,QTA_storico,FATTURATO_storico,DURATA_VOLANTINO_IN_GIORNI
0,23MA,2023-11-01,2023-11-08,BASKET TASSO ZERO,AMFACTIVEB,129.99,129.99,0,Principale,0.0,CORE WEARABLES ...,AMAZFIT,184518908.0,0.42367,0.000000,0.000000,1.0,0.000000,0,0.00,7
1,23NA,2023-12-01,2023-12-06,BASKET TASSO ZERO,AMFACTIVEB,129.99,129.99,1111,Principale,0.0,CORE WEARABLES ...,AMAZFIT,184518908.0,1.00000,0.000000,0.046811,1.0,0.756182,15,1593.97,5
2,23MA,2023-11-01,2023-11-08,BASKET TASSO ZERO,AMFACTIVEP,129.99,129.99,0,Sostitutivo,0.0,CORE WEARABLES ...,AMAZFIT,184518908.0,0.42367,0.000000,0.000000,1.0,0.000000,0,0.00,7
3,23NA,2023-12-01,2023-12-06,BASKET TASSO ZERO,AMFACTIVEP,129.99,129.99,962,Sostitutivo,0.0,CORE WEARABLES ...,AMAZFIT,184518908.0,1.00000,0.000000,0.018919,1.0,0.756182,7,745.84,5
4,23NC,2023-12-17,2023-12-24,NATALISSIMI TASSO ZERO,AMFBAND7,44.99,49.99,237,Principale,10.0,CORE WEARABLES ...,AMAZFIT,177507779.0,1.00000,0.252783,0.061233,1.0,0.068461,63,2442.52,7


In [77]:
df1['PRODUCT_GROUP'].unique()

array(['CORE WEARABLES                                    ',
       'MOBILE COMPUTING                                  ',
       'SMARTPHONES                                       ',
       'PTV/FLAT                                          ',
       'WASHINGMACHINES FREESTANDING                      '], dtype=object)

In [78]:
# DROP THE SPACES AFTER THE PRODUCT GROUP

df1.replace('CORE WEARABLES                                    ', 'CORE WEARABLES', inplace=True)
df1.replace('MOBILE COMPUTING                                  ', 'MOBILE COMPUTING', inplace=True)
df1.replace('SMARTPHONES                                       ', 'SMARTPHONES', inplace=True)
df1.replace('PTV/FLAT                                          ', 'PTV/FLAT', inplace=True)
df1.replace('WASHINGMACHINES FREESTANDING                      ', 'WASHINGMACHINES FREESTANDING', inplace=True)

In [79]:
core_wear = df1[df1['PRODUCT_GROUP']=='CORE WEARABLES']
wash = df1[df1['PRODUCT_GROUP']=='WASHINGMACHINES FREESTANDING']
pc = df1[df1['PRODUCT_GROUP']=='MOBILE COMPUTING']
smartphones = df1[df1['PRODUCT_GROUP']=='SMARTPHONES']
tv = df1[df1['PRODUCT_GROUP']=='PTV/FLAT']

In [80]:
gfk.isnull().sum()

ITEM_ID                 0
CARATTERISTICA_01       0
CARATTERISTICA_02       0
CARATTERISTICA_05      77
CARATTERISTICA_04       1
CARATTERISTICA_03       1
CARATTERISTICA_06     235
CARATTERISTICA_07    1561
dtype: int64

## SMARTPHONES

In [81]:
smartphones = df1[df1['PRODUCT_GROUP']=='SMARTPHONES']
smartphones = pd.merge(smartphones, gfk, on='ITEM_ID', how='left')
smartphones['SCONTO_PERC_MEDIO_VOLANTINO'] = smartphones.groupby('CODICE_VOLANTINO')['SCONTO_PERC'].transform('mean')

### CAR_01

In [82]:
smartphones['CARATTERISTICA_01'].unique()

array(['DISPLAY SIZE 6.1', 'DISPLAY SIZE 6.7', 'DISPLAY SIZE 6.2', nan,
       'DISPLAY SIZE 6.52', 'DISPLAY SIZE 6.6', 'DISPLAY SIZE 6.56',
       'DISPLAY SIZE 6.5', 'DISPLAY SIZE 6.59', 'DISPLAY SIZE 6.72',
       'DISPLAY SIZE 6.43', 'DISPLAY SIZE 6.74', 'DISPLAY SIZE 6.4',
       'DISPLAY SIZE 6.75', 'DISPLAY SIZE 6.79', 'DISPLAY SIZE 6.71',
       'DISPLAY SIZE 6.67', 'DISPLAY SIZE 6.28', 'DISPLAY SIZE 6.8'],
      dtype=object)

In [83]:
smartphones = smartphones[smartphones['CARATTERISTICA_01']!='TYPE GRAPHIC-DEVICE']
smartphones = smartphones.rename(columns={'CARATTERISTICA_01': 'DISPLAY_SIZE'})
smartphones['DISPLAY_SIZE'] = smartphones['DISPLAY_SIZE'].str.replace('DISPLAY SIZE ', '').astype(float)
smartphones['DISPLAY_SIZE'].fillna(smartphones['DISPLAY_SIZE'].value_counts().index[0], inplace=True)

### CAR_02

In [87]:
smartphones['CARATTERISTICA_02'].unique()

array(['OPERATOR SIM FREE', nan, 'OPERATOR WINDTRE', 'OPERATOR VODAFONE',
       'OPERATOR TIM'], dtype=object)

In [88]:
smartphones = smartphones.rename(columns={'CARATTERISTICA_02': 'OPERATOR'})
smartphones['OPERATOR'] = smartphones['OPERATOR'].str.replace('OPERATOR ', '').astype(str)
smartphones['OPERATOR'].fillna('SIM FREE', inplace=True)

### CAR_03

In [91]:
smartphones['CARATTERISTICA_03'].unique()

array(['GENERATION TOTAL* 5.X G', nan, 'GENERATION TOTAL* 4.X G'],
      dtype=object)

In the train set we had the following problem, while in the test set we did not.

There are errors because the values 'CAPACITY' should be in the next column. I then select all the rows that contain the word 'CAPACITY' in the 'CARATTERISTICA_03' column and then:

-move the values from the 'CARATTERISTICA_04' column to the 'CARATTERISTICA_05' column

-Move the values from the 'CARATTERISTICA_03' column to 'CARATTERISTICA_04' column.

In [92]:
'''
smartphones['CARATTERISTICA_03'] = smartphones['CARATTERISTICA_03'].fillna('')
righe_errate = smartphones[smartphones['CARATTERISTICA_03'].str.contains('CAPACITY')]
smartphones.loc[righe_errate.index, 'CARATTERISTICA_05'] = smartphones.loc[righe_errate.index, 'CARATTERISTICA_04']
smartphones['CARATTERISTICA_04'].replace(r'.*SIM.*', np.nan, regex=True, inplace=True)
smartphones.loc[righe_errate.index, 'CARATTERISTICA_04'] = smartphones.loc[righe_errate.index, 'CARATTERISTICA_03']
smartphones['CARATTERISTICA_03'].replace(r'.*CAPACITY.*', np.nan, regex=True, inplace=True)
smartphones['CARATTERISTICA_03'].replace('', np.nan, inplace=True)
'''

"\nsmartphones['CARATTERISTICA_03'] = smartphones['CARATTERISTICA_03'].fillna('')\nrighe_errate = smartphones[smartphones['CARATTERISTICA_03'].str.contains('CAPACITY')]\nsmartphones.loc[righe_errate.index, 'CARATTERISTICA_05'] = smartphones.loc[righe_errate.index, 'CARATTERISTICA_04']\nsmartphones['CARATTERISTICA_04'].replace(r'.*SIM.*', np.nan, regex=True, inplace=True)\nsmartphones.loc[righe_errate.index, 'CARATTERISTICA_04'] = smartphones.loc[righe_errate.index, 'CARATTERISTICA_03']\nsmartphones['CARATTERISTICA_03'].replace(r'.*CAPACITY.*', np.nan, regex=True, inplace=True)\nsmartphones['CARATTERISTICA_03'].replace('', np.nan, inplace=True)\n"

In [93]:
smartphones = smartphones.rename(columns={'CARATTERISTICA_03': 'GENERATION'})

In [94]:
# WE DEFINED A FUNCTION TO EXTRACT ONLY THE NUMBER (EX. 'GENERATION TOTAL* 4.X G' --> 4)

import re

def estrai_numero(testo):
    if isinstance(testo, str): 
        match = re.search(r'\d+\.*\d*', testo)
        if match:
            return float(match.group())
    return np.nan  

smartphones['GENERATION'] = smartphones['GENERATION'].apply(estrai_numero)


In [95]:
smartphones['GENERATION'].fillna(smartphones['GENERATION'].value_counts().index[0], inplace=True)


### CAR_04

In [96]:
smartphones['CARATTERISTICA_04'].unique()

array(['CAPACITY 2815', 'CAPACITY 3240', 'CAPACITY 3279', 'CAPACITY 4323',
       'CAPACITY 3349', 'CAPACITY 4422', 'CAPACITY 3274', 'CAPACITY 4385',
       'CAPACITY 4575', 'CAPACITY 5050', nan, 'CAPACITY 5000',
       'CAPACITY 4500', 'CAPACITY 5200', 'CAPACITY 3700', 'CAPACITY 3900',
       'CAPACITY 4700', 'CAPACITY 5010', 'CAPACITY 4300', 'CAPACITY 4820',
       'CAPACITY 3110', 'CAPACITY 4020'], dtype=object)

In [97]:
smartphones = smartphones.rename(columns={'CARATTERISTICA_04': 'CAPACITY'})
smartphones['CAPACITY'] = smartphones['CAPACITY'].str.replace('CAPACITY ', '').astype(float)
smartphones['CAPACITY'].fillna(smartphones['CAPACITY'].value_counts().index[0], inplace=True)

### CAR_05

In [100]:
smartphones['CARATTERISTICA_05'].unique()

array(['SIM CARD 1SIM SLOT +ESIM', nan, 'SIM CARD 2SIM SLOTS',
       'SIM CARD 2SIM SLOTS+ESIM'], dtype=object)

In [101]:
smartphones = smartphones.rename(columns={'CARATTERISTICA_05': 'NUMBER_OF_SIM'})
smartphones['NUMBER_OF_SIM'].replace('SIM CARD 1SIM SLOT +ESIM', 2, inplace=True)
smartphones['NUMBER_OF_SIM'].replace('SIM CARD 2SIM SLOTS', 2, inplace=True)
smartphones['NUMBER_OF_SIM'].replace('SIM CARD 2SIM SLOTS+ESIM', 3, inplace=True)
smartphones['NUMBER_OF_SIM'].fillna(smartphones['NUMBER_OF_SIM'].value_counts().index[0], inplace=True)

### CAR_06

In [104]:
smartphones['CARATTERISTICA_06'].unique()

array(['OPERATING SYST. IOS', 'OPERATING SYST. ANDROID', nan],
      dtype=object)

In [105]:
smartphones = smartphones.rename(columns={'CARATTERISTICA_06': 'OPERATING_SYST'})
smartphones['OPERATING_SYST'] = smartphones['OPERATING_SYST'].str.replace('OPERATING SYST. ', '')

def sostituisci_nan(row):
    if pd.isnull(row['OPERATING_SYST']):  
        if row['ART_COD'].startswith('APL'):  
            return 'IOS'
        else:
            return 'ANDROID'
    else:
        return row['OPERATING_SYST']  


smartphones['OPERATING_SYST'] = smartphones.apply(sostituisci_nan, axis=1)

### CAR_07

In [108]:
smartphones.drop('CARATTERISTICA_07', axis=1, inplace = True)

## CORE WEAR

In [109]:
core_wear = df1[df1['PRODUCT_GROUP']=='CORE WEARABLES']
core_wear = pd.merge(core_wear, gfk, on='ITEM_ID', how='left')
core_wear['SCONTO_PERC_MEDIO_VOLANTINO'] = core_wear.groupby('CODICE_VOLANTINO')['SCONTO_PERC'].transform('mean')

### CAR_01

In [110]:
core_wear['CARATTERISTICA_01'].unique()

array([nan, 'CATEGORY SMARTWATCHES NO SIM',
       'CATEGORY WRIST SPORT COMPUTER', 'CATEGORY SMARTWATCHES SIM',
       'CATEGORY HEALTH AND FITNESS TRACKER'], dtype=object)

In [111]:
core_wear = core_wear.rename(columns={'CARATTERISTICA_01': 'CATEGORY'})
core_wear['CATEGORY'] = core_wear['CATEGORY'].str.replace('CATEGORY ', '')
core_wear['CATEGORY'].replace(np.nan, 'NOT DEFINED', inplace=True)

### CAR_02

In [114]:
core_wear['CARATTERISTICA_02'].unique()

array([nan, 'DISPLAY TECHNOL OLED-DISPLAY', 'DISPLAY TECHNOL LCD-DISPLAY'],
      dtype=object)

In [115]:
core_wear = core_wear.rename(columns={'CARATTERISTICA_02': 'DISPLAY_QUALITY'})
core_wear['DISPLAY_QUALITY'] = core_wear['DISPLAY_QUALITY'].str.replace('DISPLAY TECHNOL ', '')
core_wear['DISPLAY_QUALITY'].replace(np.nan, 'NOT DEFINED', inplace=True)

### CAR_03

In [118]:
core_wear['CARATTERISTICA_03'].unique()

array([nan, 'DISPLAY SIZE 1.47', 'DISPLAY SIZE 1.69', 'DISPLAY SIZE 1.91',
       'DISPLAY SIZE 1.45', 'DISPLAY SIZE 1.28', 'DISPLAY SIZE 1.55',
       'DISPLAY SIZE 1.75', 'DISPLAY SIZE 1.65', 'DISPLAY SIZE 1.9',
       'DISPLAY SIZE UNKNOWN', 'DISPLAY SIZE 1.57', 'DISPLAY SIZE 1.78',
       'DISPLAY SIZE 1.58', 'DISPLAY SIZE 1.3', 'DISPLAY SIZE 1.04',
       'DISPLAY SIZE 1.41', 'DISPLAY SIZE 1.2', 'DISPLAY SIZE 1.83',
       'DISPLAY SIZE 1.4', 'DISPLAY SIZE 1.5', 'DISPLAY SIZE 1.62',
       'DISPLAY SIZE 1.39', 'DISPLAY SIZE 1.43', 'DISPLAY SIZE 0.9'],
      dtype=object)

In [119]:
core_wear = core_wear.rename(columns={'CARATTERISTICA_03': 'DISPLAY_SIZE'})
core_wear['DISPLAY_SIZE'].replace('DISPLAY SIZE UNKNOWN', np.nan, inplace=True)
core_wear['DISPLAY_SIZE'] = core_wear['DISPLAY_SIZE'].str.replace('DISPLAY SIZE ', '').astype(float)
core_wear['DISPLAY_SIZE'].fillna(core_wear['DISPLAY_SIZE'].value_counts().index[0], inplace=True)

### CAR_04

In [123]:
core_wear['CARATTERISTICA_04'].unique()

array([nan, 'WIFI NO', 'WIFI YES'], dtype=object)

In [124]:
core_wear = core_wear.rename(columns={'CARATTERISTICA_04': 'WIFI'})
core_wear['WIFI'].replace(np.nan, 0, inplace=True)
core_wear['WIFI'].replace('WIFI NO', 0, inplace=True)
core_wear['WIFI'].replace('WIFI YES', 1, inplace=True)

### CAR_05

In [126]:
core_wear['CARATTERISTICA_05'].unique()

array([nan, 'BLUETOOTH WITH BLUETOOTH'], dtype=object)

In [127]:
core_wear = core_wear.rename(columns={'CARATTERISTICA_05': 'BLUETOOTH'})
core_wear['BLUETOOTH'].replace(np.nan, 0, inplace=True)
core_wear['BLUETOOTH'].replace('BLUETOOTH WITH BLUETOOTH', 1, inplace=True)

### CAR_06 & CAR_07

In [129]:
core_wear.drop(['CARATTERISTICA_06', 'CARATTERISTICA_07'], axis=1, inplace=True)

## WASH

In [130]:
wash = df1[df1['PRODUCT_GROUP']=='WASHINGMACHINES FREESTANDING']
wash = pd.merge(wash, gfk, on='ITEM_ID', how='left')
wash['SCONTO_PERC_MEDIO_VOLANTINO'] = wash.groupby('CODICE_VOLANTINO')['SCONTO_PERC'].transform('mean')

### CAR_01

In [131]:
wash['CARATTERISTICA_01'].unique()

array(['MAIN TYPES FULLAUTOMATIC', 'MAIN TYPES WASHDRYER', nan],
      dtype=object)

In [133]:
wash = wash.rename(columns={'CARATTERISTICA_01': 'TYPE'})
wash['TYPE'].replace('MAIN TYPES FULLAUTOMATIC', 1, inplace=True)
wash['TYPE'].replace('MAIN TYPES WASHDRYER', 0, inplace=True)
wash['TYPE'].fillna(wash['TYPE'].value_counts().index[0], inplace=True)

### CAR_02

In [135]:
wash['CARATTERISTICA_02'].unique()

array(['LOADING FRONTLOADING', 'LOADING TOPLOADING', nan], dtype=object)

In [136]:
wash = wash.rename(columns={'CARATTERISTICA_02': 'TOPLOADING'})
wash['TOPLOADING'].replace('LOADING TOPLOADING', 1, inplace=True)
wash['TOPLOADING'].replace('LOADING FRONTLOADING', 0, inplace=True)
wash['TOPLOADING'].fillna(wash['TOPLOADING'].value_counts().index[0], inplace=True)

### CAR_03

In [138]:
wash['CARATTERISTICA_03'].unique()

array(['DEPTH IN CM STANDARD > 48 CM', 'DEPTH IN CM SLIM <= 48 CM', nan],
      dtype=object)

In [139]:
wash = wash.rename(columns={'CARATTERISTICA_03': 'DEPTH_CM>48'})
wash['DEPTH_CM>48'].replace('DEPTH IN CM STANDARD > 48 CM', 1, inplace=True)
wash['DEPTH_CM>48'].replace('DEPTH IN CM SLIM <= 48 CM', 0, inplace=True)
wash['DEPTH_CM>48'].fillna(wash['DEPTH_CM>48'].value_counts().index[0], inplace=True)


### CAR_04

In [141]:
wash['CARATTERISTICA_04'].unique()

array(['21EN.EFF.CLASS A', '21EN.EFF.CLASS B', '21EN.EFF.CLASS C', nan,
       '21EN.EFF.CLASS D'], dtype=object)

In [142]:
wash = wash.rename(columns={'CARATTERISTICA_04': 'ENERGY_CLASS'})

In [145]:
wash['ENERGY_CLASS'].replace('21EN.EFF.CLASS D', 3, inplace=True)
wash['ENERGY_CLASS'].replace('21EN.EFF.CLASS C', 4, inplace=True)
wash['ENERGY_CLASS'].replace('21EN.EFF.CLASS B', 5, inplace=True)
wash['ENERGY_CLASS'].replace('21EN.EFF.CLASS A', 6, inplace=True)
wash['ENERGY_CLASS'].fillna(wash['ENERGY_CLASS'].value_counts().index[0], inplace=True)

### CAR_05

In [147]:
wash['CARATTERISTICA_05'].unique()

array(['LOADING KG 8', 'LOADING KG 9', 'LOADING KG 7', 'LOADING KG 6',
       nan, 'LOADING KG 12', 'LOADING KG 10', 'LOADING KG 11'],
      dtype=object)

In [148]:
wash = wash.rename(columns={'CARATTERISTICA_05': 'LOADING_KG'})
wash['LOADING_KG'] = wash['LOADING_KG'].str.replace('LOADING KG ', '').astype(float)
wash['LOADING_KG'].fillna(wash['LOADING_KG'].value_counts().index[0], inplace=True)

### CAR_06

In [151]:
wash['CARATTERISTICA_06'].unique()

array(['SMART CONNECT NO', nan, 'SMART CONNECT SMART CONNECT'],
      dtype=object)

In [152]:
wash = wash.rename(columns={'CARATTERISTICA_06': 'SMART_CONNECT'})
wash['SMART_CONNECT'].replace(np.nan, 0, inplace=True)
wash['SMART_CONNECT'].replace('SMART CONNECT NO', 0, inplace=True)
wash['SMART_CONNECT'].replace('SMART CONNECT SMART CONNECT', 1, inplace=True)

### CAR_07

In [154]:
wash.drop('CARATTERISTICA_07', axis=1, inplace=True)

## PC

In [155]:
pc = df1[df1['PRODUCT_GROUP']=='MOBILE COMPUTING']
pc = pd.merge(pc, gfk, on='ITEM_ID', how='left')
pc['SCONTO_PERC_MEDIO_VOLANTINO'] = pc.groupby('CODICE_VOLANTINO')['SCONTO_PERC'].transform('mean')


### CAR_01

In [156]:
pc['CARATTERISTICA_01'].unique()

array(['GPU MODEL RAD 610M', nan, 'GPU MODEL UHD GRAPH. 600',
       'GPU MODEL RAD', 'GPU MODEL APPLE 8CORE GPU',
       'GPU MODEL APPLE 10COREGPU', 'GPU MODEL INTEL IRIS XE',
       'GPU MODEL INTEL UHD GPU', 'GPU MODEL GEF RTX3050',
       'GPU MODEL GEF RTX3050 TI', 'GPU MODEL MALI-G52 2EE MC',
       'GPU MODEL GEF RTX4050', 'GPU MODEL GEF RTX4060',
       'GPU MODEL APPLE 7CORE GPU'], dtype=object)

In [159]:
pc['CARATTERISTICA_01'] = pc['CARATTERISTICA_01'].str.replace('GPU MODEL ', '')

In [160]:
pc['CARATTERISTICA_01'].fillna('NOT DEFINED', inplace=True)

INTEL produce: UHD, HD, IRIS, MALI-G72, ADRENO506

With the following loop, we add the word 'INTEL' before the gpu model produced by them


In [161]:
for i in range(pc.shape[0]):
    if 'UHD' in pc.iloc[i, 21] and 'INTEL' not in pc.iloc[i, 21]:
        pc.iloc[i, 21] = 'INTEL'+' '+pc.iloc[i, 21]
    elif 'HD' in pc.iloc[i, 21] and 'INTEL' not in pc.iloc[i, 21]:
        pc.iloc[i, 21] = 'INTEL'+' '+pc.iloc[i, 21]
    elif 'IRIS' in pc.iloc[i, 21] and 'INTEL' not in pc.iloc[i, 21]:
        pc.iloc[i, 21] = 'INTEL'+' '+pc.iloc[i, 21]
    elif 'MALI' in pc.iloc[i, 21] and 'INTEL' not in pc.iloc[i, 21]:
        pc.iloc[i, 21] = 'INTEL'+' '+pc.iloc[i, 21]
    elif 'ADRENO506' in pc.iloc[i, 21] and 'INTEL' not in pc.iloc[i, 21]:
        pc.iloc[i, 21] = 'INTEL'+' '+pc.iloc[i, 21]

In [162]:
pc = pc.rename(columns={'CARATTERISTICA_01': 'GPU_MODEL'})

In [163]:
pc['GPU_MODEL'].value_counts()

GPU_MODEL
INTEL IRIS XE            26
NOT DEFINED              23
APPLE 8CORE GPU          15
INTEL UHD GPU            12
RAD                      10
APPLE 7CORE GPU           8
APPLE 10COREGPU           6
GEF RTX4060               5
RAD 610M                  4
INTEL UHD GRAPH. 600      4
GEF RTX3050               4
GEF RTX3050 TI            1
INTEL MALI-G52 2EE MC     1
GEF RTX4050               1
Name: count, dtype: int64

In [164]:
pc['GPU_MODEL'].unique()

array(['RAD 610M', 'NOT DEFINED', 'INTEL UHD GRAPH. 600', 'RAD',
       'APPLE 8CORE GPU', 'APPLE 10COREGPU', 'INTEL IRIS XE',
       'INTEL UHD GPU', 'GEF RTX3050', 'GEF RTX3050 TI',
       'INTEL MALI-G52 2EE MC', 'GEF RTX4050', 'GEF RTX4060',
       'APPLE 7CORE GPU'], dtype=object)

In [165]:
# WITH THIS LOOP, WE CREATE NEW COLUMNS IN ORDER TO ENCODE THE GPU MODELS BASED ON THEIR VERSION

INTEL = []
APPLE = []
RAD = []
GEF = [] 
NOT_DEFINED = []

for i in range(pc.shape[0]):
    if 'NOT DEFINED' == pc.iloc[i, 21]:
        NOT_DEFINED.append(1)
        INTEL.append(0)
        APPLE.append(0)
        RAD.append(0)
        GEF.append(0)
    elif 'INTEL' in pc.iloc[i, 21]:
        NOT_DEFINED.append(0)
        INTEL.append(1)
        APPLE.append(0)
        RAD.append(0)
        GEF.append(0)
    elif 'APPLE' in pc.iloc[i, 21]:
        NOT_DEFINED.append(0)
        INTEL.append(0)
        APPLE.append(1)
        RAD.append(0)
        GEF.append(0)
    elif 'RAD' in pc.iloc[i, 21]:
        NOT_DEFINED.append(0)
        INTEL.append(0)
        APPLE.append(0)
        RAD.append(1)
        GEF.append(0)
    elif 'GEF' in pc.iloc[i, 21]:
        NOT_DEFINED.append(0)
        INTEL.append(0)
        APPLE.append(0)
        RAD.append(0)
        GEF.append(1)

pc['VERSION_NOT_DEFINED'] = NOT_DEFINED
pc['VERSION_INTEL'] = INTEL
pc['VERSION_APPLE'] = APPLE
pc['VERSION_RAD'] = RAD
pc['VERSION_GEF'] = GEF

In [166]:
pc.head()

,CODICE_VOLANTINO,DATA_INIZIO,DATA_FINE,NOME_CAMPAGNA,ART_COD,PREZZO_PROMO,PREZZO_LISTINO,STOCK_PZ,TIPOLOGIA_PRODOTTO,SCONTO_PERC,PRODUCT_GROUP,BRAND,ITEM_ID,KPI_1,KPI_2,KPI_3,KPI_4,KPI_5,QTA_storico,FATTURATO_storico,DURATA_VOLANTINO_IN_GIORNI,GPU_MODEL,CARATTERISTICA_02,CARATTERISTICA_05,CARATTERISTICA_04,CARATTERISTICA_03,CARATTERISTICA_06,CARATTERISTICA_07,SCONTO_PERC_MEDIO_VOLANTINO,VERSION_NOT_DEFINED,VERSION_INTEL,VERSION_APPLE,VERSION_RAD,VERSION_GEF
0,23MB,2023-11-09,2023-11-15,BLACK FRIDAY 1,ACEA31524PR2KM,449.9,649.9,1112,Principale,30.77,MOBILE COMPUTING,ACER,180577047.0,0.960614,0.698699,0.658007,1.000000,1.000000,791,297913.02,6,RAD 610M,PROCESSOR RYZEN 5,OS VERSION WIN11 HOME,STORAGE IN GB 512,RAM IN GB 8,DISPLAY SIZE 15.6,CONVERTIBLE NO,29.019286,0,0,0,1,0
1,23NB,2023-12-07,2023-12-16,SOTTOCOSTO TASSO ZERO,ACEA5155758Y8,549.0,699.9,220,Principale,21.56,MOBILE COMPUTING,ACER,185137535.0,1.000000,0.000000,0.000000,0.000000,0.000000,0,0.00,9,NOT DEFINED,NaN,NaN,NaN,NaN,NaN,NaN,27.256923,1,0,0,0,0
2,23NA,2023-12-01,2023-12-06,BASKET TASSO ZERO,ACEA5155758Y8,599.0,699.9,0,Principale,14.42,MOBILE COMPUTING,ACER,185137535.0,1.000000,0.000000,0.000000,0.000000,0.000000,0,0.00,5,NOT DEFINED,NaN,NaN,NaN,NaN,NaN,NaN,19.625000,1,0,0,0,0
3,23MA,2023-11-01,2023-11-08,BASKET TASSO ZERO,ACECB3153HC322,249.9,289.9,0,Principale,13.80,MOBILE COMPUTING,ACER,178878940.0,0.960614,0.226019,0.000000,0.143809,0.000000,0,0.00,7,INTEL UHD GRAPH. 600,PROCESSOR CELERON N,OS VERSION CHROME OS,STORAGE IN GB 64,RAM IN GB 4,DISPLAY SIZE 15.6,CONVERTIBLE NO,17.554706,0,1,0,0,0
4,23NB,2023-12-07,2023-12-16,SOTTOCOSTO TASSO ZERO,ACESF31443R90U,649.9,829.9,572,Principale,21.69,MOBILE COMPUTING,ACER,184819894.0,1.000000,0.000000,0.075439,0.321116,0.705751,26,13523.66,9,RAD,PROCESSOR RYZEN 7,OS VERSION WIN11 HOME,STORAGE IN GB 512,RAM IN GB 16,DISPLAY SIZE 14,CONVERTIBLE NO,27.256923,0,0,0,1,0


FOCUS ON INTEL

high_quality_models = ['IRIS', 'ADRENO']                                                -->3

medium_quality_models = ['UHD', 'GRAPH. 6']                                             -->2

low_quality_models = ['HD', 'GRAPH. 5', 'GRAPH. 4', 'ADRENO506', 'MALI']            -->1

In [167]:
pc['GPU_MODEL'][pc['GPU_MODEL'].str.contains('INTEL', na=False)].unique()

array(['INTEL UHD GRAPH. 600', 'INTEL IRIS XE', 'INTEL UHD GPU',
       'INTEL MALI-G52 2EE MC'], dtype=object)

FOCUS ON APPLE

high_quality_models = ['16', '14']                                                -->3

medium_quality_models = ['10']                                             -->2

low_quality_models = ['8', '7']            -->1

In [168]:
pc['GPU_MODEL'][pc['GPU_MODEL'].str.contains('APPLE', na=False)].unique()

array(['APPLE 8CORE GPU', 'APPLE 10COREGPU', 'APPLE 7CORE GPU'],
      dtype=object)

FOCUS ON RAD

high_quality_models = ['VEGA']                                                -->3

medium_quality_models = ['PRO', '610', 'R7', 'R5']                                             -->2

low_quality_models = ['R3', 'R2']            -->1

In [169]:
pc['GPU_MODEL'][pc['GPU_MODEL'].str.contains('RAD', na=False)].unique()

array(['RAD 610M', 'RAD'], dtype=object)

FOCUS ON GEF

high_quality_models = ['RTX']                                                -->3

medium_quality_models = ['GTX']                                             -->2

low_quality_models = ['MX']            -->1

In [170]:
pc['GPU_MODEL'][pc['GPU_MODEL'].str.contains('GEF', na=False)].unique()

array(['GEF RTX3050', 'GEF RTX3050 TI', 'GEF RTX4050', 'GEF RTX4060'],
      dtype=object)

In [171]:
# WE CREATE A NEW COLUMN 'QUALITY VERSION' THAT ASSIGN A VALUE BETWEEN 1 AND 3 BASED ON THE QUALITY OF THE GPU

QUALITY_VERSION = []
for i in range(pc.shape[0]):
    if 'INTEL' in pc.iloc[i, 21]:
        if 'IRIS' in pc.iloc[i, 21] or 'ADRENO' in pc.iloc[i, 21]:
            QUALITY_VERSION.append(3)
        elif 'UHD' in pc.iloc[i, 21] or 'GRAPH. 6' in pc.iloc[i, 21]:
            QUALITY_VERSION.append(2)
        else:
            QUALITY_VERSION.append(1)

    elif 'APPLE' in pc.iloc[i, 21]:
        if '16' in pc.iloc[i, 21] or '14' in pc.iloc[i, 21]:
            QUALITY_VERSION.append(3)
        elif '10' in pc.iloc[i, 21]:
            QUALITY_VERSION.append(2)
        else:
            QUALITY_VERSION.append(1)
        
    elif 'RAD' in pc.iloc[i, 21]:
        if 'VEGA' in pc.iloc[i, 21]:
            QUALITY_VERSION.append(3)
        elif 'PRO' in pc.iloc[i, 21] or '610' in pc.iloc[i, 21] or 'R7' in pc.iloc[i, 21] or 'R5' in pc.iloc[i, 21]:
            QUALITY_VERSION.append(2)
        else:
            QUALITY_VERSION.append(1)

    elif 'GEF' in pc.iloc[i, 21]:
        if 'RTX' in pc.iloc[i, 21]:
            QUALITY_VERSION.append(3)
        elif 'GTX' in pc.iloc[i, 21]:
            QUALITY_VERSION.append(2)
        else:
            QUALITY_VERSION.append(1)
    
    else:  #SE IL VALORE è NOT DEFINED
         QUALITY_VERSION.append(1)
    

pc['QUALITY_VERSION'] = QUALITY_VERSION
    


In [172]:
pc.drop('GPU_MODEL', axis=1, inplace=True)

### CAR_02

In [174]:
pc['CARATTERISTICA_02'].unique()

array(['PROCESSOR RYZEN 5', nan, 'PROCESSOR CELERON N',
       'PROCESSOR RYZEN 7', 'PROCESSOR APPLE M-SERIES',
       'PROCESSOR CORE I5', 'PROCESSOR CORE I7', 'PROCESSOR CORE I3',
       'PROCESSOR KOMPANIO', 'PROCESSOR RYZEN 3',
       'PROCESSOR PENTIUM GOLD'], dtype=object)

In [175]:
pc = pc.rename(columns={'CARATTERISTICA_02': 'PROCESSOR'})

In [176]:
pc['PROCESSOR'] = pc['PROCESSOR'].str.replace('PROCESSOR ', '')
pc['PROCESSOR'].fillna('NOT DEFINED', inplace=True)

WE PERFORMED ORDINAL ENCODING ORDING TO THE QUALITY OF THE PROCESSORS 

quality_high = ['CORE I7', 'RYZEN 7', 'CORE I5', 'RYZEN 5'] -->3

quality_medium = ['CORE I3', 'RYZEN 3', 'A9-SERIES', 'A12-SERIES', 'PENTIUM GOLD', 'PENTIUM SILVER', 'APPLE'] -->2

quality_low = tutti gli altri  -->1

In [177]:
QUALITY_PROCESSOR = []

for i in range(pc.shape[0]):
    if 'CORE I7' in pc.iloc[i,21] or 'RYZEN 7' in pc.iloc[i,21] or 'CORE I5' in pc.iloc[i,21] or 'RYZEN 5' in pc.iloc[i,21]:
        QUALITY_PROCESSOR.append(3)
    elif 'CORE I3' in pc.iloc[i,21] or 'RYZEN 3' in pc.iloc[i,21] or 'A9-SERIES' in pc.iloc[i,21] or 'A12-SERIES' in pc.iloc[i,21] or 'PENTIUM GOLD' in pc.iloc[i,21] or 'PENTIUM SILVER' in pc.iloc[i,21] or 'APPLE' in pc.iloc[i,21]:
        QUALITY_PROCESSOR.append(2)
    else:
        QUALITY_PROCESSOR.append(1)

pc['QUALITY_PROCESSOR'] = QUALITY_PROCESSOR

In [178]:
pc.drop('PROCESSOR', axis=1, inplace=True)

### CAR_03

In [179]:
pc['CARATTERISTICA_03'].unique()

array(['RAM IN GB 8', nan, 'RAM IN GB 4', 'RAM IN GB 16'], dtype=object)

In [180]:
pc = pc.rename(columns={'CARATTERISTICA_03': 'RAM_GB'})
pc['RAM_GB'] = pc['RAM_GB'].str.replace('RAM IN GB ', '').astype(float)
pc['RAM_GB'].fillna(pc['RAM_GB'].value_counts().index[0], inplace=True)

### CAR_04

In [182]:
pc['CARATTERISTICA_04'].unique()

array(['STORAGE IN GB 512', nan, 'STORAGE IN GB 64', 'STORAGE IN GB 256',
       'STORAGE IN GB 128', 'STORAGE IN GB 1000'], dtype=object)

In [183]:
pc = pc.rename(columns={'CARATTERISTICA_04': 'STORAGE_GB'})
pc['STORAGE_GB'] = pc['STORAGE_GB'].str.replace('STORAGE IN GB ', '').astype(float)
pc['STORAGE_GB'].fillna(pc['STORAGE_GB'].value_counts().index[0], inplace=True)

### CAR_05

In [185]:
pc['CARATTERISTICA_05'].unique()

array(['OS VERSION WIN11 HOME', nan, 'OS VERSION CHROME OS',
       'OS VERSION MACOS MONTEREY', 'OS VERSION MACOS VENTURA',
       'OS VERSION WIN11 HOME S', 'OS VERSION MACOS BIG SUR'],
      dtype=object)

In [186]:
pc = pc.rename(columns={'CARATTERISTICA_05': 'OS_VERSION'})
pc['OS_VERSION'] = pc['OS_VERSION'].str.split('OS VERSION ').str.get(1).str.split().str.get(0)
pc['OS_VERSION'].fillna('NOT DEFINED', inplace=True)

### CAR_06

In [189]:
pc['CARATTERISTICA_06'].unique()

array(['DISPLAY SIZE 15.6', nan, 'DISPLAY SIZE 14', 'DISPLAY SIZE 13.6',
       'DISPLAY SIZE 13.3', 'DISPLAY SIZE 15.3', 'DISPLAY SIZE 16',
       'DISPLAY SIZE 17.3', 'DISPLAY SIZE 14.5', 'DISPLAY SIZE 13'],
      dtype=object)

In [190]:
pc = pc.rename(columns={'CARATTERISTICA_06': 'DISPLAY_SIZE'})
pc['DISPLAY_SIZE'] = pc['DISPLAY_SIZE'].str.replace('DISPLAY SIZE ', '').astype(float)
pc['DISPLAY_SIZE'].fillna(pc['DISPLAY_SIZE'].value_counts().index[0], inplace=True)

### CAR_07

In [192]:
pc['CARATTERISTICA_07'].unique()

array(['CONVERTIBLE NO', nan, 'CONVERTIBLE YES'], dtype=object)

In [193]:
pc = pc.rename(columns={'CARATTERISTICA_07': 'CONVERTIBLE'})
pc['CONVERTIBLE'].replace(np.nan, 0, inplace=True)
pc['CONVERTIBLE'].replace('CONVERTIBLE NO', 0, inplace=True)
pc['CONVERTIBLE'].replace('CONVERTIBLE YES', 1, inplace=True)

## TV

In [195]:
tv = df1[df1['PRODUCT_GROUP']=='PTV/FLAT']
tv = pd.merge(tv, gfk, on='ITEM_ID', how='left')
tv['SCONTO_PERC_MEDIO_VOLANTINO'] = tv.groupby('CODICE_VOLANTINO')['SCONTO_PERC'].transform('mean')


### CAR_01

In [196]:
tv['CARATTERISTICA_01'].unique()

array(['DISPLAY SIZE 43', 'DISPLAY SIZE 55', 'DISPLAY SIZE 65', nan,
       'DISPLAY SIZE 75', 'DISPLAY SIZE 77', 'DISPLAY SIZE 40',
       'DISPLAY SIZE 50', 'DISPLAY SIZE 32', 'DISPLAY SIZE 54.6',
       'DISPLAY SIZE 48'], dtype=object)

In [197]:
tv = tv.rename(columns={'CARATTERISTICA_01': 'DISPLAY_SIZE'})
tv['DISPLAY_SIZE'] = tv['DISPLAY_SIZE'].str.replace('DISPLAY SIZE ', '').astype(float)
tv['DISPLAY_SIZE'].fillna(tv['DISPLAY_SIZE'].value_counts().index[0], inplace=True)

### CAR_02

In [199]:
tv['CARATTERISTICA_02'].unique()

array(['HD TYPES ULTRA HD/4K (3840X2160P)', nan, 'HD TYPES FULL HD',
       'HD TYPES HD READY'], dtype=object)

In [200]:
tv = tv.rename(columns={'CARATTERISTICA_02': 'HD_QUALITY'})
tv['HD_QUALITY'].replace('HD TYPES HD READY', 1, inplace=True)
tv['HD_QUALITY'].replace('HD TYPES FULL HD', 2, inplace=True)
tv['HD_QUALITY'].replace('HD TYPES ULTRA HD/4K (3840X2160P)', 3, inplace=True)
tv['HD_QUALITY'].fillna(tv['HD_QUALITY'].value_counts().index[0], inplace=True)

### CAR_03

In [203]:
tv['CARATTERISTICA_03'].unique()

array(['MFM NO MFM', nan], dtype=object)

In [204]:
tv = tv.rename(columns={'CARATTERISTICA_03': 'MFM'})
tv['MFM'].replace('MFM NO MFM', 0, inplace=True)
tv['MFM'].fillna(0, inplace=True)

### CAR_04

In [206]:
tv['CARATTERISTICA_04'].unique()

array(['SMART TV SMART TV', nan], dtype=object)

In the train set we had the following problem, while in the test set we did not.

The rows were is present 'CURVED CURVED' or 'CURVED NOT CURVED', have the information abuot 'smart' on the next column.

Since the information about 'curve' is available just for a few observation, we decided to don't care about it.

So, for these rows we moved all the values on the previous column. Similarly to what we did for 'CARATTERISTICA_03' of smartphone.

In [207]:
'''
selezione = (tv['CARATTERISTICA_04'] == 'CURVED CURVED') | (tv['CARATTERISTICA_04'] == 'CURVED NOT CURVED')

tv.loc[selezione, 'CARATTERISTICA_04'] = tv.loc[selezione, 'CARATTERISTICA_05']
tv.loc[selezione, 'CARATTERISTICA_05'] = tv.loc[selezione, 'CARATTERISTICA_06']

tv['CARATTERISTICA_04'] = tv['CARATTERISTICA_04'].fillna('')
righe_errate = tv[tv['CARATTERISTICA_04'].str.contains('CURVED')]
tv.loc[righe_errate.index, 'CARATTERISTICA_04'] = tv.loc[righe_errate.index, 'CARATTERISTICA_05']
tv['CARATTERISTICA_05'].replace(r'.*SMART.*', np.nan, regex=True, inplace=True)
tv.loc[righe_errate.index, 'CARATTERISTICA_05'] = tv.loc[righe_errate.index, 'CARATTERISTICA_06']
tv['CARATTERISTICA_06'].replace(r'.*DISPLAY.*', np.nan, regex=True, inplace=True)
tv['CARATTERISTICA_04'].replace('', 0, inplace=True)
'''

"\nselezione = (tv['CARATTERISTICA_04'] == 'CURVED CURVED') | (tv['CARATTERISTICA_04'] == 'CURVED NOT CURVED')\n\ntv.loc[selezione, 'CARATTERISTICA_04'] = tv.loc[selezione, 'CARATTERISTICA_05']\ntv.loc[selezione, 'CARATTERISTICA_05'] = tv.loc[selezione, 'CARATTERISTICA_06']\n"

In [209]:
tv = tv.rename(columns={'CARATTERISTICA_04': 'SMART'})
tv['SMART'].replace('SMART TV SMART TV', 1, inplace=True)
tv['SMART'].replace('SMART TV NO SMART TV', 0, inplace=True)
tv['SMART'].fillna(tv['SMART'].value_counts().index[0], inplace=True)

### CAR_05

In [212]:
tv['CARATTERISTICA_05'].unique()

array(['DISPLAY TECHNOL LCD-DISPLAY', 'DISPLAY TECHNOL OLED-DISPLAY', nan],
      dtype=object)

In [213]:
tv = tv.rename(columns={'CARATTERISTICA_05': 'OLED'})
tv['OLED'].replace('DISPLAY TECHNOL LCD-DISPLAY', 0, inplace=True)
tv['OLED'].replace('DISPLAY TECHNOL OLED-DISPLAY', 1, inplace=True)
tv['OLED'].fillna(tv['OLED'].value_counts().index[0], inplace=True)

### CAR_06

In [216]:
tv['CARATTERISTICA_06'].unique()

array(['WCG QUANTUM DOT', 'WCG VARIOUS WCG TEC', nan, 'WCG NO WCG'],
      dtype=object)

In [217]:
tv = tv.rename(columns={'CARATTERISTICA_06': 'WCG'})
tv['WCG'].replace('WCG NO WCG', 0, inplace=True)
tv['WCG'].replace('WCG VARIOUS WCG TEC', 1, inplace=True)
tv['WCG'].replace('WCG QUANTUM DOT', 2, inplace=True)
tv['WCG'].fillna(tv['WCG'].value_counts().index[0], inplace=True)

### CAR_07

In [220]:
tv['CARATTERISTICA_07'].unique()

array(['MINILED BACKLIT NO MINILED', nan, 'MINILED BACKLIT MINILED'],
      dtype=object)

In [221]:
tv = tv.rename(columns={'CARATTERISTICA_07': 'MINILED'})
tv['MINILED'].replace('MINILED BACKLIT NO MINILED', 0, inplace=True)
tv['MINILED'].replace('MINILED BACKLIT MINILED', 1, inplace=True)
tv['MINILED'].fillna(tv['MINILED'].value_counts().index[0], inplace=True)

# ULTERIORI MODIFICHE AI DATASET

In [224]:
# WE DEFINED THIS FUNCTION IN ORDER TO CREATE A NEW COLUMN 'SCONTO_PERC_MEDIO_NOME_CAMPAGNA'.
# FIRST OF ALL WE ORGANISED 'NOME_CAMPAGNA' in 11 CATEGORIES: back_to_school, basket, black_friday, fuori_tutto, passione_casa, sottocosto, 
# tasso_zero, special_days, fly, bike, 50% and other.
# THEN, FOR EACH CATEGORY WE CALCULATED THE AVERAGE DISCOUNT OF THE PRODUCTS
# WE WILL APPLY THIS FUNCTION TO EACH PRODUCT GROUP DATAFRAME

def nomi_campagna(df):
    
    pattern = ['BACK TO SCHOOL', 'BACK TO SCHOOL 1', 'BACK TO SCHOOL 2', 'Back to School', 'Back to School continua']
    df.NOME_CAMPAGNA.replace(pattern, 'back_to_school', inplace = True)

    pattern = ['BASKET', 'BASKET INTERESSI ZERO + TRADE IN', 'BASKET MARZO', 'BASKET SCONTI APRILE', 'BASKET SCONTI FEBBRAIO', 'BASKET SCONTI FEBBRAIO 2']
    df.NOME_CAMPAGNA.replace(pattern, 'basket', inplace = True)

    pattern = ['BLACK FRIDAY - EPISODIO 1', 'BLACK FRIDAY - EPISODIO 3', 'BLACK FRIDAY EP.0', 'BLACK FRIDAY EP.1',
            'BLACK FRIDAY EP.2', 'Bastard Black Friday', 'Black Addams Friday', 'Black Addams Friday Continua', 'Black Rock Friday',
            'Change Black Friday', 'Change Black Friday 2', 'Change Black Friday Gran Finale', 'Change Black Friday', 
            'Manà Manà Black Friday', 'Manà Manà Black Friday continua','Manà Manà Black Friday continua 2', 'Manà Manà Black Friday continua 3',
            'BF 2 - TEMPTATION BLACK FRIDAY', 'SUMMER BLACK FRIDAY 2', 'SUMMER BLACK FRIDAY 1', 'Summer Black Friday', 'Summer Black Friday 2']
    df.NOME_CAMPAGNA.replace(pattern, 'black_friday', inplace = True)

    pattern = ['Continua il Fuoritutto Gennaio', 'FUORITUTTO 1','FUORITUTTO 1 AGOSTO','FUORITUTTO 2','FUORITUTTO 2 AGOSTO','Fuoritutto',
            'Fuoritutto 1','Fuoritutto 1 Summer','Fuoritutto 2','Fuoritutto 2 Summer','Fuoritutto Continua','Fuoritutto Gennaio','Fuoritutto Inverno 1',
            'Fuoritutto Inverno 2','Fuoritutto1 E','Fuoritutto1 W','Fuoritutto2 E','Fuoritutto2 W','Il Vero Fuoritutto', 'Il Vero Fuoritutto continua']
    df.NOME_CAMPAGNA.replace(pattern, 'fuori_tutto', inplace = True)

    pattern = ['PASSIONE CASA','PASSIONE CASA - BUONI INNOVAZIONE','Passione Casa','Passione Casa - Hitachi Abbinato',
            'Passione Casa - Kitchenaid Abbinato','Passione Casa Settembre','Passione casa']
    df.NOME_CAMPAGNA.replace(pattern, 'passione_casa', inplace = True)

    pattern = ['SOTTOCOSTO', 'SOTTOCOSTO LUGLIO', 'SOTTOCOSTO NATALE', 'Sottoprezzo','Sottocosto Natale','Sottocosto Luglio','Sottocosto Giugno',
            'Sottocosto Dicembre','Sottocosto Autunno','Sottocosto 2', 'Sottocosto', "Sotto Costo un mare d'offerte"]
    df.NOME_CAMPAGNA.replace(pattern, 'sottocosto', inplace = True)

    pattern = ['Happy Sconti - Tasso 0', 'Tasso 0 Risparmio Vero',"Tasso 0 - Sconti d'Estate",'Tasso 0 - Sconti', 'TASSO ZERO - IT TRADE IN','TASSO ZERO',
            'TASSO 0 - SPECIALE PULIZIA'," Solo il Meglio? E' scontato - Tasso Zero", 'Settembre di Sconti - Tasso 0', 'UN MARE DI SCONTI A TASSO ZERO',
            "Sconti di Settembre Tasso Zero", "Sconti d'Estate a Tasso Zero", "Sconti che Sognavi - Tasso Zero", 'SCONTI DI MAGGIO A TASSO ZERO', 
            'Risparmio Vero Tasso Zero', 'Natale Tasso Zero', 'NATALE A TASSO ZERO','Mare di Sconti - Tasso 0','Happy Sconti - Tasso 0',
            "Solo il Meglio? E' scontato - Tasso Zero"]
    df.NOME_CAMPAGNA.replace(pattern, 'tasso_zero', inplace = True)

    pattern = ['Natalissimi','Halloween Days', 'Primavera di Sconti', 'Fioriscono gli Sconti', 'Collezione primavera', 'CALENDARIO AVVENTO', 'HALLOWEEN WEEK',
            'Summer Days', 'I Natalissimi', 'Buon anno con Unieuro']
    df.NOME_CAMPAGNA.replace(pattern, 'special_days', inplace = True)

    pattern = ['Flyer Volo', 'VOLAGRATIS', 'Vola con noi']
    df.NOME_CAMPAGNA.replace(pattern, 'fly', inplace = True)

    pattern = ['BIKE IN REGALO', 'Flyer Bike', 'Promo Bike WOW']
    df.NOME_CAMPAGNA.replace(pattern, 'bike', inplace = True)

    pattern = ['Sconti Fino Al 50 - Focus Ped', 'SECONDO AL 50', 'Flyer Secondo Al 50%', 'Sconti Fino Al 50 - Focus Bianco']
    df.NOME_CAMPAGNA.replace(pattern, '50%', inplace = True)

    pattern = ['VOLANTINO BUONI','Ogni momento è buono','TRADE IN PC','Lovely Days',
            'Unieuro ti regala la Mia','Amore al primo acquisto', 'Tanto ti dà Tanto','Cookeo connect',
            'Se Vuoi Puoi','Tra tante offerte la tua','Se Vuoi Puoi 2','E amore al primo acquisto','Noi ci siamo', 'Sconti batticuore']

    df.NOME_CAMPAGNA.replace(pattern, 'other', inplace = True)

    df['SCONTO_PERC_MEDIO_NOME_CAMPAGNA'] = df.groupby('NOME_CAMPAGNA')['SCONTO_PERC'].transform('mean')

## SMARTPHONES

In [225]:
smartphones['DATA_INIZIO'] = pd.to_datetime(smartphones['DATA_INIZIO'])
smartphones['DATA_FINE'] = pd.to_datetime(smartphones['DATA_FINE'])

smartphones['Month'] = smartphones['DATA_INIZIO'].dt.month
smartphones['Month'] = smartphones['Month'].astype(str)

smartphones['Year'] = smartphones['DATA_INIZIO'].dt.year
smartphones['Year'].replace([2018, 2019], 'PRE_COVID', inplace=True)
smartphones['Year'].replace([2020, 2021], 'COVID', inplace=True)
smartphones['Year'].replace([2022, 2023], 'POST_COVID', inplace=True)

smartphones['OPERATOR'].fillna('SIM FREE', inplace=True)
smartphones['OPERATOR'].replace('WIND', 'WINDTRE', inplace=True)

In [226]:
nomi_campagna(smartphones)

From the test set data, we divided the months into three bands according to their degree of profitability. The first band is assigned value 1, the second band value 2 and the third band value 3.

As regards smartphones:

November --> 2 

December --> 3 

In [227]:
smartphones['Month'].replace(['11'], 2, inplace=True)
smartphones['Month'].replace(['12'], 3, inplace=True)
smartphones.rename(columns={'Month': 'Month_Redditivity'}, inplace=True)

In [228]:
smartphones['OPERATING_SYST'].replace('IOS', 1, inplace=True)
smartphones['OPERATING_SYST'].replace('ANDROID', 0, inplace=True)

## PC

In [229]:
pc['DATA_INIZIO'] = pd.to_datetime(pc['DATA_INIZIO'])
pc['DATA_FINE'] = pd.to_datetime(pc['DATA_FINE'])

pc['Month'] = pc['DATA_INIZIO'].dt.month
pc['Month'] = pc['Month'].astype(str)

pc['Year'] = pc['DATA_INIZIO'].dt.year
pc['Year'].replace([2018, 2019], 'PRE_COVID', inplace=True)
pc['Year'].replace([2020, 2021], 'COVID', inplace=True)
pc['Year'].replace([2022, 2023], 'POST_COVID', inplace=True)

In [230]:
nomi_campagna(pc)

From the test set data, we divided the months into three bands according to their degree of profitability. The first band is assigned value 1, the second band value 2 and the third band value 3.

As regards pc:

November --> 2  

December --> 3 

In [231]:
pc['Month'].replace(['11'], 2, inplace=True)
pc['Month'].replace(['12'], 3, inplace=True)
pc.rename(columns={'Month': 'Month_Redditivity'}, inplace=True)

## WASH

In [232]:
wash['DATA_INIZIO'] = pd.to_datetime(wash['DATA_INIZIO'])
wash['DATA_FINE'] = pd.to_datetime(wash['DATA_FINE'])

wash['Month'] = wash['DATA_INIZIO'].dt.month
wash['Month'] = wash['Month'].astype(str)

wash['Year'] = wash['DATA_INIZIO'].dt.year
wash['Year'].replace([2018, 2019], 'PRE_COVID', inplace=True)
wash['Year'].replace([2020, 2021], 'COVID', inplace=True)
wash['Year'].replace([2022, 2023], 'POST_COVID', inplace=True)

In [233]:
nomi_campagna(wash)

From the test set data, we divided the months into three bands according to their degree of profitability. The first band is assigned value 1, the second band value 2 and the third band value 3.

As regards wash:

November --> 2  

December --> 2 

In [234]:
wash['Month'].replace(['11'], 2, inplace=True)
wash['Month'].replace(['12'], 2, inplace=True)
wash.rename(columns={'Month': 'Month_Redditivity'}, inplace=True)

## CORE WEAR

In [235]:
core_wear['DATA_INIZIO'] = pd.to_datetime(core_wear['DATA_INIZIO'])
core_wear['DATA_FINE'] = pd.to_datetime(core_wear['DATA_FINE'])

core_wear['Month'] = core_wear['DATA_INIZIO'].dt.month
core_wear['Month'] = core_wear['Month'].astype(str)

core_wear['Year'] = core_wear['DATA_INIZIO'].dt.year
core_wear['Year'].replace([2018, 2019], 'PRE_COVID', inplace=True)
core_wear['Year'].replace([2020, 2021], 'COVID', inplace=True)
core_wear['Year'].replace([2022, 2023], 'POST_COVID', inplace=True)

In [236]:
nomi_campagna(core_wear)

From the test set data, we divided the months into three bands according to their degree of profitability. The first band is assigned value 1, the second band value 2 and the third band value 3.

As regards core_wear:

November --> 2 fascia 

December --> 5 fascia

We decided to allocate 5 to December because it is extremely more profitable for this product group than the other months

In [237]:
core_wear['Month'].replace(['11'], 2, inplace=True)
core_wear['Month'].replace(['12'], 5, inplace=True)
core_wear.rename(columns={'Month': 'Month_Redditivity'}, inplace=True)

In [238]:
core_wear['DISPLAY_QUALITY'].replace('NOT DEFINED', np.nan, inplace=True)
core_wear['DISPLAY_QUALITY'].replace('ELECTRONIC INK', 1, inplace=True)
core_wear['DISPLAY_QUALITY'].replace('LCD-DISPLAY', 2, inplace=True)
core_wear['DISPLAY_QUALITY'].replace('OLED-DISPLAY', 3, inplace=True)
core_wear['DISPLAY_QUALITY'].fillna(round(core_wear['DISPLAY_QUALITY'].mean(), 2), inplace=True)


## TV

In [239]:
tv['DATA_INIZIO'] = pd.to_datetime(tv['DATA_INIZIO'])
tv['DATA_FINE'] = pd.to_datetime(tv['DATA_FINE'])

tv['Month'] = tv['DATA_INIZIO'].dt.month
tv['Month'] = tv['Month'].astype(str)

tv['Year'] = tv['DATA_INIZIO'].dt.year
tv['Year'].replace([2018, 2019], 'PRE_COVID', inplace=True)
tv['Year'].replace([2020, 2021], 'COVID', inplace=True)
tv['Year'].replace([2022, 2023], 'POST_COVID', inplace=True)

In [240]:
nomi_campagna(tv)

From the test set data, we divided the months into three bands according to their degree of profitability. The first band is assigned value 1, the second band value 2 and the third band value 3.

As regards tv:

November --> 2  

December --> 2 

In [241]:
tv['Month'].replace(['11'], 2, inplace=True)
tv['Month'].replace(['12'], 2, inplace=True)
tv.rename(columns={'Month': 'Month_Redditivity'}, inplace=True)

# SALVATAGGIO DATASET PER TEST

In [242]:
#smartphones.to_csv('/Users/vincenzocamerlengo/Desktop/Data Science in Action/Unieuro Dataset-20240305/dati_luiss_2/test/smartphone_test.csv', index=False)
#pc.to_csv('/Users/vincenzocamerlengo/Desktop/Data Science in Action/Unieuro Dataset-20240305/dati_luiss_2/test/pc_test.csv', index=False)
#wash.to_csv('/Users/vincenzocamerlengo/Desktop/Data Science in Action/Unieuro Dataset-20240305/dati_luiss_2/test/wash_test.csv', index=False)
#core_wear.to_csv('/Users/vincenzocamerlengo/Desktop/Data Science in Action/Unieuro Dataset-20240305/dati_luiss_2/test/core_wear_test.csv', index=False)
#tv.to_csv('/Users/vincenzocamerlengo/Desktop/Data Science in Action/Unieuro Dataset-20240305/dati_luiss_2/test/tv_test.csv', index=False)

# MODELLI

## SMARTPHONES

In [23]:
smartphones_test = pd.read_csv('./Working Code/Datasets/smartphone_test.csv')
smartphones_test['DATA_INIZIO'] = pd.to_datetime(smartphones_test['DATA_INIZIO'])
smartphones_test['OPERATOR'].fillna('SIM FREE', inplace=True)

smartphones_train = pd.read_csv('./Working Code/Datasets/smartphones1.csv')
smartphones_train['DATA_INIZIO'] = pd.to_datetime(smartphones_train['DATA_INIZIO'])
smartphones_train = smartphones_train[smartphones_train['DATA_INIZIO'].dt.year >= 2022].reset_index()
smartphones_train.drop('index', axis=1, inplace=True)

In [24]:
X_test = smartphones_test[['PREZZO_PROMO', 'PREZZO_LISTINO', 'STOCK_PZ',
                 'TIPOLOGIA_PRODOTTO', 'SCONTO_PERC', 'KPI_1',
                 'KPI_2', 'KPI_3', 'KPI_4', 'KPI_5', 'QTA_storico',
                 'FATTURATO_storico', 'DURATA_VOLANTINO_IN_GIORNI',
                 'DISPLAY_SIZE', 'OPERATOR', 'NUMBER_OF_SIM',
                 'CAPACITY', 'GENERATION', 'OPERATING_SYST', 'Month_Redditivity',
                 'SCONTO_PERC_MEDIO_VOLANTINO', 'SCONTO_PERC_MEDIO_NOME_CAMPAGNA']]

from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse = False)
oheTransform = ohe.fit_transform(X_test[['TIPOLOGIA_PRODOTTO', 'OPERATOR']])
oheTransform = pd.DataFrame(oheTransform, columns=ohe.get_feature_names_out())

final_df = pd.concat([X_test, oheTransform], axis=1)

final_df = final_df.drop(['TIPOLOGIA_PRODOTTO', 'OPERATOR'], axis = 1)

X_test = final_df.copy()
TIPOLOGIA_PRODOTTO_ND = [0] * len(X_test)
X_test.insert(20, 'TIPOLOGIA_PRODOTTO_ND', TIPOLOGIA_PRODOTTO_ND)

X_train = smartphones_train[['PREZZO_PROMO', 'PREZZO_LISTINO', 'STOCK_PZ',
                 'TIPOLOGIA_PRODOTTO', 'SCONTO_PERC', 'KPI_1',
                 'KPI_2', 'KPI_3', 'KPI_4', 'KPI_5', 'QTA_storico',
                 'FATTURATO_storico', 'DURATA_VOLANTINO_IN_GIORNI',
                 'DISPLAY_SIZE', 'OPERATOR', 'NUMBER_OF_SIM',
                 'CAPACITY', 'GENERATION', 'OPERATING_SYST', 'Month_Redditivity',
                 'SCONTO_PERC_MEDIO_VOLANTINO', 'SCONTO_PERC_MEDIO_NOME_CAMPAGNA']]

from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse = False)
oheTransform = ohe.fit_transform(X_train[['TIPOLOGIA_PRODOTTO', 'OPERATOR']])
oheTransform = pd.DataFrame(oheTransform, columns=ohe.get_feature_names_out())

final_df = pd.concat([X_train, oheTransform], axis=1)

final_df = final_df.drop(['TIPOLOGIA_PRODOTTO', 'OPERATOR'], axis = 1)

X_train = final_df.copy()

y_train = smartphones_train[['QTA']].values

Our best model for the smartphones is the Linear Regression

In [27]:
from sklearn.preprocessing import StandardScaler
column_names = X_train.columns.tolist()
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

from sklearn.linear_model import LinearRegression
classifier = LinearRegression()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)
y_pred = np.where(y_pred < 0, 0, y_pred)


In [45]:
smartphones_forecast = pd.DataFrame({'CODICE_VOLANTINO' : smartphones_test['CODICE_VOLANTINO'],
                                     'ART_COD' : smartphones_test['ART_COD'],
                                     'FORECAST' : y_pred.flatten().tolist()})

## PC

In [47]:
pc_test = pd.read_csv('./Working Code/Datasets/pc_test.csv')
pc_test['DATA_INIZIO'] = pd.to_datetime(pc_test['DATA_INIZIO'])

pc_train = pd.read_csv('./Working Code/Datasets/pc1.csv')
pc_train['DATA_INIZIO'] = pd.to_datetime(pc_train['DATA_INIZIO'])
pc_train = pc_train[pc_train['DATA_INIZIO'].dt.year >= 2022].reset_index()
pc_train.drop('index', axis=1, inplace=True)

In [48]:
X_test = pc_test[['PREZZO_PROMO', 'PREZZO_LISTINO', 'STOCK_PZ',
        'TIPOLOGIA_PRODOTTO','SCONTO_PERC', 'KPI_1',
        'KPI_2', 'KPI_3', 'KPI_4', 'KPI_5', 'QTA_storico',
        'FATTURATO_storico', 'DURATA_VOLANTINO_IN_GIORNI',
        'STORAGE_GB', 'RAM_GB', 'DISPLAY_SIZE', 'CONVERTIBLE',
        'SCONTO_PERC_MEDIO_VOLANTINO', 'VERSION_NOT_DEFINED',
        'VERSION_INTEL', 'VERSION_APPLE', 'VERSION_RAD',
        'VERSION_GEF', 'QUALITY_VERSION', 'QUALITY_PROCESSOR',
        'Month_Redditivity', 'SCONTO_PERC_MEDIO_NOME_CAMPAGNA']]


from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse = False)
oheTransform = ohe.fit_transform(X_test[['TIPOLOGIA_PRODOTTO']])
oheTransform = pd.DataFrame(oheTransform, columns=ohe.get_feature_names_out())

final_df = pd.concat([X_test, oheTransform], axis=1)

final_df = final_df.drop(['TIPOLOGIA_PRODOTTO'], axis = 1)

X_test = final_df.copy()
TIPOLOGIA_PRODOTTO_ND = [0] * len(X_test)
X_test.insert(26, 'TIPOLOGIA_PRODOTTO_ND', TIPOLOGIA_PRODOTTO_ND)

X_train = pc_train[['PREZZO_PROMO', 'PREZZO_LISTINO', 'STOCK_PZ',
        'TIPOLOGIA_PRODOTTO','SCONTO_PERC', 'KPI_1',
        'KPI_2', 'KPI_3', 'KPI_4', 'KPI_5', 'QTA_storico',
        'FATTURATO_storico', 'DURATA_VOLANTINO_IN_GIORNI',
        'STORAGE_GB', 'RAM_GB', 'DISPLAY_SIZE', 'CONVERTIBLE',
        'SCONTO_PERC_MEDIO_VOLANTINO', 'VERSION_NOT_DEFINED',
        'VERSION_INTEL', 'VERSION_APPLE', 'VERSION_RAD',
        'VERSION_GEF', 'QUALITY_VERSION', 'QUALITY_PROCESSOR',
        'Month_Redditivity', 'SCONTO_PERC_MEDIO_NOME_CAMPAGNA']]


from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse = False)
oheTransform = ohe.fit_transform(X_train[['TIPOLOGIA_PRODOTTO']])
oheTransform = pd.DataFrame(oheTransform, columns=ohe.get_feature_names_out())

final_df = pd.concat([X_train, oheTransform], axis=1)

final_df = final_df.drop(['TIPOLOGIA_PRODOTTO'], axis = 1)

X_train = final_df.copy()

y_train = pc_train[['QTA']].values

Our best model is an XGBoost

In [49]:
hyperparameters = {'subsample': 0.9, 'n_estimators': 400, 'min_child_weight': 1, 'max_depth': 5, 'learning_rate': 0.01, 'gamma': 0.2, 'colsample_bytree': 1.0}

import xgboost as xgb

xgbr = xgb.XGBRegressor(random_state=69, **hyperparameters)
xgbr.fit(X_train, y_train)
y_pred = xgbr.predict(X_test)
y_pred = np.where(y_pred < 0, 0, y_pred)

In [50]:
pc_forecast = pd.DataFrame({'CODICE_VOLANTINO' : pc_test['CODICE_VOLANTINO'],
                                     'ART_COD' : pc_test['ART_COD'],
                                     'FORECAST' : y_pred})

## WASH

In [52]:
wash_test = pd.read_csv('./Working Code/Datasets/wash_test.csv')
wash_test['DATA_INIZIO'] = pd.to_datetime(wash_test['DATA_INIZIO'])

wash_train = pd.read_csv('./Working Code/Datasets/wash1.csv')
wash_train['DATA_INIZIO'] = pd.to_datetime(wash_train['DATA_INIZIO'])
wash_train = wash_train[wash_train['DATA_INIZIO'].dt.year >= 2022].reset_index()
wash_train.drop('index', axis=1, inplace=True)

In [53]:
X_test = wash_test[['PREZZO_PROMO', 'PREZZO_LISTINO', 'STOCK_PZ',
        'TIPOLOGIA_PRODOTTO','SCONTO_PERC', 'KPI_1',
        'KPI_2', 'KPI_3', 'KPI_4', 'KPI_5', 'QTA_storico',
        'FATTURATO_storico', 'DURATA_VOLANTINO_IN_GIORNI',
        'TYPE', 'TOPLOADING', 'LOADING_KG', 'ENERGY_CLASS',
        'DEPTH_CM>48', 'SMART_CONNECT', 'SCONTO_PERC_MEDIO_VOLANTINO', 'Month_Redditivity',
        'SCONTO_PERC_MEDIO_NOME_CAMPAGNA']]

from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse = False)
oheTransform = ohe.fit_transform(X_test[['TIPOLOGIA_PRODOTTO']])
oheTransform = pd.DataFrame(oheTransform, columns=ohe.get_feature_names_out())

final_df = pd.concat([X_test, oheTransform], axis=1)

final_df = final_df.drop(['TIPOLOGIA_PRODOTTO'], axis = 1)

X_test = final_df.copy()
TIPOLOGIA_PRODOTTO_ND = [0] * len(X_test)
TIPOLOGIA_PRODOTTO_Sostitutivo = [0] * len(X_test)
X_test.insert(21, 'TIPOLOGIA_PRODOTTO_ND', TIPOLOGIA_PRODOTTO_ND)
X_test.insert(23, 'TIPOLOGIA_PRODOTTO_Sostitutivo', TIPOLOGIA_PRODOTTO_Sostitutivo)


X_train = wash_train[['PREZZO_PROMO', 'PREZZO_LISTINO', 'STOCK_PZ',
        'TIPOLOGIA_PRODOTTO','SCONTO_PERC', 'KPI_1',
        'KPI_2', 'KPI_3', 'KPI_4', 'KPI_5', 'QTA_storico',
        'FATTURATO_storico', 'DURATA_VOLANTINO_IN_GIORNI',
        'TYPE', 'TOPLOADING', 'LOADING_KG', 'ENERGY_CLASS',
        'DEPTH_CM>48', 'SMART_CONNECT', 'SCONTO_PERC_MEDIO_VOLANTINO', 'Month_Redditivity',
        'SCONTO_PERC_MEDIO_NOME_CAMPAGNA']]

from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse = False)
oheTransform = ohe.fit_transform(X_train[['TIPOLOGIA_PRODOTTO']])
oheTransform = pd.DataFrame(oheTransform, columns=ohe.get_feature_names_out())

final_df = pd.concat([X_train, oheTransform], axis=1)

final_df = final_df.drop(['TIPOLOGIA_PRODOTTO'], axis = 1)

X_train = final_df.copy()

y_train = wash_train[['QTA']].values

Our best model is a Random Forest

In [54]:
hyperparameters = {'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 40, 'bootstrap': False}

from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor(random_state=69, **hyperparameters)
rfr.fit(X_train, y_train)
y_pred = rfr.predict(X_test)
y_pred = np.where(y_pred < 0, 0, y_pred)

In [55]:
wash_forecast = pd.DataFrame({'CODICE_VOLANTINO' : wash_test['CODICE_VOLANTINO'],
                                     'ART_COD' : wash_test['ART_COD'],
                                     'FORECAST' : y_pred})

## CORE WEAR

In [57]:
core_wear_test = pd.read_csv('./Working Code/Datasets/core_wear_test.csv')
core_wear_test['DATA_INIZIO'] = pd.to_datetime(core_wear_test['DATA_INIZIO'])

core_wear_train = pd.read_csv('./Working Code/Datasets/core_wear1.csv')
core_wear_train['DATA_INIZIO'] = pd.to_datetime(core_wear_train['DATA_INIZIO'])
core_wear_train = core_wear_train[core_wear_train['DATA_INIZIO'].dt.year >= 2022].reset_index()
core_wear_train.drop('index', axis=1, inplace=True)

In [58]:
X_test = core_wear_test[['PREZZO_PROMO', 'PREZZO_LISTINO', 'STOCK_PZ',
        'TIPOLOGIA_PRODOTTO','SCONTO_PERC', 'KPI_1',
        'KPI_2', 'KPI_3', 'KPI_4', 'KPI_5', 'QTA_storico',
        'FATTURATO_storico', 'DURATA_VOLANTINO_IN_GIORNI',
        'DISPLAY_QUALITY', 'BLUETOOTH', 'WIFI', 'DISPLAY_SIZE',
        'SCONTO_PERC_MEDIO_VOLANTINO', 'Month_Redditivity', 'SCONTO_PERC_MEDIO_NOME_CAMPAGNA']]

from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse = False)
oheTransform = ohe.fit_transform(X_test[['TIPOLOGIA_PRODOTTO']])
oheTransform = pd.DataFrame(oheTransform, columns=ohe.get_feature_names_out())

final_df = pd.concat([X_test, oheTransform], axis=1)

final_df = final_df.drop(['TIPOLOGIA_PRODOTTO'], axis = 1)

X_test = final_df.copy()
TIPOLOGIA_PRODOTTO_ND = [0] * len(X_test)
X_test.insert(19, 'TIPOLOGIA_PRODOTTO_ND', TIPOLOGIA_PRODOTTO_ND)



X_train = core_wear_train[['PREZZO_PROMO', 'PREZZO_LISTINO', 'STOCK_PZ',
        'TIPOLOGIA_PRODOTTO','SCONTO_PERC', 'KPI_1',
        'KPI_2', 'KPI_3', 'KPI_4', 'KPI_5', 'QTA_storico',
        'FATTURATO_storico', 'DURATA_VOLANTINO_IN_GIORNI',
        'DISPLAY_QUALITY', 'BLUETOOTH', 'WIFI', 'DISPLAY_SIZE',
        'SCONTO_PERC_MEDIO_VOLANTINO', 'Month_Redditivity', 'SCONTO_PERC_MEDIO_NOME_CAMPAGNA']]

from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse = False)
oheTransform = ohe.fit_transform(X_train[['TIPOLOGIA_PRODOTTO']])
oheTransform = pd.DataFrame(oheTransform, columns=ohe.get_feature_names_out())

final_df = pd.concat([X_train, oheTransform], axis=1)

final_df = final_df.drop(['TIPOLOGIA_PRODOTTO'], axis = 1)

X_train = final_df.copy()

y_train = core_wear_train[['QTA']].values

Our best model is an XGBoost

In [59]:
hyperparameters = {'subsample': 0.6, 'n_estimators': 500, 'min_child_weight': 3, 'max_depth': 9, 'learning_rate': 0.05, 'gamma': 0, 'colsample_bytree': 0.9}

import xgboost as xgb

xgbr = xgb.XGBRegressor(random_state=69, **hyperparameters)
xgbr.fit(X_train, y_train)
y_pred = xgbr.predict(X_test)
y_pred = np.where(y_pred < 0, 0, y_pred)

In [60]:
core_wear_forecast = pd.DataFrame({'CODICE_VOLANTINO' : core_wear_test['CODICE_VOLANTINO'],
                                     'ART_COD' : core_wear_test['ART_COD'],
                                     'FORECAST' : y_pred})

## TV

In [62]:
tv_test = pd.read_csv('./Working Code/Datasets/tv_test.csv')
tv_test['DATA_INIZIO'] = pd.to_datetime(tv_test['DATA_INIZIO'])

tv_train = pd.read_csv('./Working Code/Datasets/tv1.csv')
tv_train['DATA_INIZIO'] = pd.to_datetime(tv_train['DATA_INIZIO'])
tv_train = tv_train[tv_train['DATA_INIZIO'].dt.year >= 2022].reset_index()
tv_train.drop('index', axis=1, inplace=True)

In [63]:
X_test = tv_test[['PREZZO_PROMO', 'PREZZO_LISTINO', 'STOCK_PZ',
        'SCONTO_PERC', 'KPI_1',
        'KPI_2', 'KPI_3', 'KPI_4', 'KPI_5', 'QTA_storico',
        'FATTURATO_storico', 'DURATA_VOLANTINO_IN_GIORNI',
        'DISPLAY_SIZE', 'HD_QUALITY', 'OLED', 'SMART',
        'MFM', 'WCG', 'MINILED', 'SCONTO_PERC_MEDIO_VOLANTINO', 'Month_Redditivity',
        'SCONTO_PERC_MEDIO_NOME_CAMPAGNA']]

X_train = tv_train[['PREZZO_PROMO', 'PREZZO_LISTINO', 'STOCK_PZ',
        'SCONTO_PERC', 'KPI_1',
        'KPI_2', 'KPI_3', 'KPI_4', 'KPI_5', 'QTA_storico',
        'FATTURATO_storico', 'DURATA_VOLANTINO_IN_GIORNI',
        'DISPLAY_SIZE', 'HD_QUALITY', 'OLED', 'SMART',
        'MFM', 'WCG', 'MINILED', 'SCONTO_PERC_MEDIO_VOLANTINO', 'Month_Redditivity',
        'SCONTO_PERC_MEDIO_NOME_CAMPAGNA']]

y_train = tv_train[['QTA']].values


Our best model is an XGBoost

In [64]:
hyperparameters = {'subsample': 0.6, 'n_estimators': 300, 'min_child_weight': 2, 'max_depth': 5, 'learning_rate': 0.01, 'gamma': 0, 'colsample_bytree': 0.6}

import xgboost as xgb

xgbr = xgb.XGBRegressor(random_state=69, **hyperparameters)
xgbr.fit(X_train, y_train)
y_pred = xgbr.predict(X_test)
y_pred = np.where(y_pred < 0, 0, y_pred)

In [65]:
tv_forecast = pd.DataFrame({'CODICE_VOLANTINO' : tv_test['CODICE_VOLANTINO'],
                                     'ART_COD' : tv_test['ART_COD'],
                                     'FORECAST' : y_pred})

# OUR FORECAST

In [67]:
forecast = pd.concat([smartphones_forecast, pc_forecast, wash_forecast, core_wear_forecast, tv_forecast], ignore_index=True)

In [68]:
forecast.shape

(1382, 3)

In [69]:
#forecast.to_csv('/Users/vincenzocamerlengo/Desktop/Data Science in Action/Unieuro Dataset-20240305/dati_luiss_2/test/FORECAST.csv', index=False)